![QuantConnect Logo](https://cdn.quantconnect.com/web/i/icon.png)
<hr>

# Volatility Trading Research Notebook

## Options Strategies with Ensemble Regime Detection

This notebook implements a comprehensive volatility trading research framework for QuantConnect.

**Key Features:**
- Multiple volatility models with PCA-based aggregate forecasting
- Ensemble regime detection (GMM, K-Means, Agglomerative Clustering, Change-Point Detection)
- Six options strategies with delta optimization
- Stop loss optimization per strategy and regime
- Vectorized computations (no multiprocessing)

**Test Ticker:** HIMS (ticker-agnostic design for future portfolio expansion)

---

## Section 1: Initial Setup & Structure

### 1.1 Import Libraries and Configure Environment

In [ ]:
# ============================================================================
# SECTION 1: INITIAL SETUP & STRUCTURE
# ============================================================================

# Standard libraries
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from typing import Dict, List, Tuple, Optional, Union
import json

# Scientific computing
from scipy import stats
from scipy.optimize import minimize
from scipy.signal import find_peaks

# Machine Learning - Clustering
from sklearn.mixture import GaussianMixture
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

# Change-Point Detection
try:
    import ruptures as rpt
    RUPTURES_AVAILABLE = True
except ImportError:
    RUPTURES_AVAILABLE = False
    print("Warning: ruptures library not available. Install with: pip install ruptures")

# Visualization
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.colors import LinearSegmentedColormap
import seaborn as sns

try:
    import plotly.graph_objects as go
    import plotly.express as px
    from plotly.subplots import make_subplots
    PLOTLY_AVAILABLE = True
except ImportError:
    PLOTLY_AVAILABLE = False
    print("Warning: plotly not available for interactive charts")

# Set plotting defaults
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = [14, 6]
plt.rcParams['font.size'] = 12
plt.rcParams['axes.titlesize'] = 14
plt.rcParams['axes.labelsize'] = 12

# QuantConnect Research Environment
qb = QuantBook()

print("Libraries imported successfully.")
print(f"NumPy version: {np.__version__}")
print(f"Pandas version: {pd.__version__}")
print(f"Ruptures available: {RUPTURES_AVAILABLE}")
print(f"Plotly available: {PLOTLY_AVAILABLE}")

### 1.2 Color Palette Definitions

**CRITICAL:** These color mappings must be used consistently across ALL visualizations throughout the notebook.

#### Strategy Colors
| Strategy | Color | Hex Code |
|----------|-------|----------|
| Calendar Spread | Blue | #1f77b4 |
| Double Diagonal | Orange | #ff7f0e |
| Straddle | Green | #2ca02c |
| Strangle | Red | #d62728 |
| Bull Put Spread | Purple | #9467bd |
| Bull Call Spread | Brown | #8c564b |

#### Regime Colors
| Regime | Color | Hex Code |
|--------|-------|----------|
| Bull_Low_Vol | Green | #2ecc71 |
| Bull_High_Vol | Orange | #f39c12 |
| Bear | Red | #e74c3c |
| Choppy | Gray | #95a5a6 |

In [ ]:
# ============================================================================
# COLOR PALETTE DEFINITIONS
# ============================================================================

# Strategy color mapping - USE CONSISTENTLY ACROSS ALL VISUALIZATIONS
STRATEGY_COLORS = {
    'Calendar Spread': '#1f77b4',    # Blue
    'Double Diagonal': '#ff7f0e',    # Orange
    'Straddle': '#2ca02c',           # Green
    'Strangle': '#d62728',           # Red
    'Bull Put Spread': '#9467bd',    # Purple
    'Bull Call Spread': '#8c564b',   # Brown
}

# Regime color mapping - USE CONSISTENTLY ACROSS ALL VISUALIZATIONS
REGIME_COLORS = {
    'Bull_Low_Vol': '#2ecc71',       # Green
    'Bull_High_Vol': '#f39c12',      # Orange
    'Bear': '#e74c3c',               # Red
    'Choppy': '#95a5a6',             # Gray
    'Sideways': '#95a5a6',           # Gray (alias)
    'Unknown': '#bdc3c7',            # Light Gray
}

# Volatility regime colors
VOL_REGIME_COLORS = {
    'Low': '#3498db',                # Light Blue
    'Normal': '#2ecc71',             # Green
    'Elevated': '#f39c12',           # Orange
    'Extreme': '#e74c3c',            # Red
}

# Helper functions for consistent color access
def get_strategy_color(strategy_name: str) -> str:
    """Get consistent color for a strategy."""
    return STRATEGY_COLORS.get(strategy_name, '#7f7f7f')  # Default gray

def get_regime_color(regime_name: str) -> str:
    """Get consistent color for a regime."""
    return REGIME_COLORS.get(regime_name, '#bdc3c7')  # Default light gray

def get_vol_regime_color(vol_regime: str) -> str:
    """Get consistent color for a volatility regime."""
    return VOL_REGIME_COLORS.get(vol_regime, '#bdc3c7')

# Display color palettes
print("Strategy Colors:")
for strategy, color in STRATEGY_COLORS.items():
    print(f"  {strategy}: {color}")

print("\nRegime Colors:")
for regime, color in REGIME_COLORS.items():
    print(f"  {regime}: {color}")

print("\nVolatility Regime Colors:")
for regime, color in VOL_REGIME_COLORS.items():
    print(f"  {regime}: {color}")

### 1.3 Configuration Parameters

In [ ]:
# ============================================================================
# CONFIGURATION PARAMETERS
# ============================================================================

# Primary ticker for analysis (ticker-agnostic design)
TICKER = 'HIMS'

# Market and sector proxies
MARKET_PROXY = 'SPY'
SECTOR_MAPPING = {
    'Technology': 'XLK',
    'Healthcare': 'XLV',
    'Financial': 'XLF',
    'Consumer Discretionary': 'XLY',
    'Consumer Staples': 'XLP',
    'Energy': 'XLE',
    'Utilities': 'XLU',
    'Materials': 'XLB',
    'Industrials': 'XLI',
    'Real Estate': 'XLRE',
    'Communication Services': 'XLC',
}

# Default sector ETF (HIMS is healthcare)
SECTOR_ETF = 'XLV'

# Date range for analysis (minimum 2-3 years recommended)
START_DATE = datetime(2022, 1, 1)
END_DATE = datetime(2024, 12, 31)

# Volatility calculation windows
VOL_WINDOWS = [5, 10, 20, 30]

# PCA configuration
PCA_LAG_PERIODS = [1, 2, 3, 5, 10, 20]
PCA_N_COMPONENTS = 3

# Regime detection configuration
N_REGIMES_RANGE = range(3, 7)  # Test 3-6 regimes
DEFAULT_N_REGIMES = 4

# Options strategy configurations
CALENDAR_DELTAS = [0.50, 0.40, 0.30, 0.20]
DIAGONAL_DELTA_PAIRS = [(0.30, -0.30), (0.25, -0.25), (0.20, -0.20)]
STRANGLE_DELTA_PAIRS = [(0.30, -0.30), (0.25, -0.25), (0.20, -0.20), (0.16, -0.16)]
BULL_PUT_SHORT_DELTAS = [-0.30, -0.20, -0.16, -0.10]
BULL_CALL_LONG_DELTAS = [0.50, 0.40, 0.30]
SPREAD_WIDTHS = [5, 10]  # Strike width for spreads

# Stop loss thresholds to test
STOP_LOSS_THRESHOLDS = [-0.10, -0.15, -0.20, -0.25, -0.30, -0.40, -0.50, None]  # None = no stop

# Rolling window for metrics
ROLLING_SHARPE_WINDOW = 60

# Minimum figure sizes
FIG_SIZE_TIMESERIES = (14, 6)
FIG_SIZE_REGIME = (14, 10)
FIG_SIZE_COMPARISON = (14, 8)
FIG_SIZE_HEATMAP = (10, 8)
FIG_SIZE_DASHBOARD = (22, 16)
FIG_SIZE_ROLLING = (12, 5)

print(f"Configuration loaded for ticker: {TICKER}")
print(f"Analysis period: {START_DATE.date()} to {END_DATE.date()}")
print(f"Volatility windows: {VOL_WINDOWS}")
print(f"Testing {len(STOP_LOSS_THRESHOLDS)} stop loss thresholds")

### 1.4 Data Fetching Functions

In [ ]:
# ============================================================================
# DATA FETCHING FUNCTIONS
# ============================================================================

def fetch_equity_data(ticker: str, start_date: datetime, end_date: datetime, 
                      resolution: str = 'Daily') -> pd.DataFrame:
    """
    Fetch historical OHLCV data for an equity.
    
    Parameters:
    -----------
    ticker : str
        Stock ticker symbol
    start_date : datetime
        Start date for data fetch
    end_date : datetime
        End date for data fetch
    resolution : str
        Data resolution ('Daily', 'Hour', 'Minute')
        
    Returns:
    --------
    pd.DataFrame
        Historical OHLCV data
    """
    # Add equity to universe
    symbol = qb.AddEquity(ticker, Resolution.Daily).Symbol
    
    # Fetch historical data
    history = qb.History(symbol, start_date, end_date, Resolution.Daily)
    
    if history.empty:
        print(f"Warning: No data available for {ticker}")
        return pd.DataFrame()
    
    # Reset index and clean up
    df = history.reset_index()
    df = df.rename(columns={'time': 'Date'})
    df = df.set_index('Date')
    
    # Calculate returns
    df['returns'] = df['close'].pct_change()
    df['log_returns'] = np.log(df['close'] / df['close'].shift(1))
    
    return df


def fetch_options_chain(ticker: str, date: datetime) -> pd.DataFrame:
    """
    Fetch options chain data for a specific date.
    
    Parameters:
    -----------
    ticker : str
        Underlying stock ticker
    date : datetime
        Date to fetch options chain
        
    Returns:
    --------
    pd.DataFrame
        Options chain with strikes, prices, Greeks, etc.
    """
    # Add option universe
    equity = qb.AddEquity(ticker, Resolution.Daily)
    option = qb.AddOption(ticker, Resolution.Daily)
    
    # Set option filter
    option.SetFilter(-10, 10, timedelta(days=0), timedelta(days=90))
    
    # Get option chain
    chain = qb.OptionChainProvider.GetOptionContractList(equity.Symbol, date)
    
    if not chain:
        return pd.DataFrame()
    
    # Build options dataframe
    options_data = []
    for contract in chain:
        try:
            # Get contract details
            history = qb.History(contract, date, date + timedelta(days=1), Resolution.Daily)
            if not history.empty:
                row = {
                    'symbol': str(contract),
                    'strike': contract.ID.StrikePrice,
                    'expiry': contract.ID.Date,
                    'option_type': 'Call' if contract.ID.OptionRight == OptionRight.Call else 'Put',
                    'bid': history['bidprice'].iloc[-1] if 'bidprice' in history.columns else np.nan,
                    'ask': history['askprice'].iloc[-1] if 'askprice' in history.columns else np.nan,
                    'last': history['close'].iloc[-1] if 'close' in history.columns else np.nan,
                    'volume': history['volume'].iloc[-1] if 'volume' in history.columns else np.nan,
                }
                options_data.append(row)
        except Exception as e:
            continue
    
    return pd.DataFrame(options_data)


def get_options_chain(ticker: str, date: datetime) -> pd.DataFrame:
    """Alias for fetch_options_chain for API consistency."""
    return fetch_options_chain(ticker, date)


def select_options_by_delta(chain: pd.DataFrame, target_delta: float, 
                            option_type: str = 'Call') -> pd.DataFrame:
    """
    Select options contracts closest to target delta.
    
    Parameters:
    -----------
    chain : pd.DataFrame
        Options chain data
    target_delta : float
        Target delta value (positive for calls, negative for puts)
    option_type : str
        'Call' or 'Put'
        
    Returns:
    --------
    pd.DataFrame
        Filtered options closest to target delta
    """
    if chain.empty or 'delta' not in chain.columns:
        return pd.DataFrame()
    
    # Filter by option type
    filtered = chain[chain['option_type'] == option_type].copy()
    
    if filtered.empty:
        return pd.DataFrame()
    
    # Calculate distance from target delta
    filtered['delta_distance'] = np.abs(filtered['delta'] - target_delta)
    
    # Sort by delta distance and return closest
    return filtered.nsmallest(1, 'delta_distance')


print("Data fetching functions defined.")

### 1.5 Fetch Historical Data

In [ ]:
# ============================================================================
# FETCH HISTORICAL DATA
# ============================================================================

import time
start_time = time.time()

print(f"Fetching data for {TICKER}...")

# Fetch underlying equity data
df_asset = fetch_equity_data(TICKER, START_DATE, END_DATE)
print(f"  {TICKER}: {len(df_asset)} days of data")

# Fetch market proxy data
df_market = fetch_equity_data(MARKET_PROXY, START_DATE, END_DATE)
print(f"  {MARKET_PROXY}: {len(df_market)} days of data")

# Fetch sector ETF data
df_sector = fetch_equity_data(SECTOR_ETF, START_DATE, END_DATE)
print(f"  {SECTOR_ETF}: {len(df_sector)} days of data")

# Align all dataframes to common dates
common_dates = df_asset.index.intersection(df_market.index).intersection(df_sector.index)
df_asset = df_asset.loc[common_dates]
df_market = df_market.loc[common_dates]
df_sector = df_sector.loc[common_dates]

elapsed = time.time() - start_time
print(f"\nData fetch completed in {elapsed:.2f} seconds")
print(f"Common trading days: {len(common_dates)}")
print(f"Date range: {common_dates[0]} to {common_dates[-1]}")

---

## Section 2: Volatility Modeling & Comparison

This section implements multiple volatility models and creates a PCA-based aggregate forecast.

### 2.1 True Realized Volatility Calculation

In [ ]:
# ============================================================================
# SECTION 2: VOLATILITY MODELING & COMPARISON
# ============================================================================

def calculate_volatility(prices: pd.Series, returns: pd.Series = None, 
                         method: str = 'historical', window: int = 20,
                         annualize: bool = True) -> pd.Series:
    """
    Calculate volatility using various methods.
    
    Parameters:
    -----------
    prices : pd.Series
        Price series (for range-based methods)
    returns : pd.Series
        Return series (for return-based methods)
    method : str
        Volatility calculation method
    window : int
        Rolling window size
    annualize : bool
        Whether to annualize volatility
        
    Returns:
    --------
    pd.Series
        Volatility series
    """
    annualization_factor = np.sqrt(252) if annualize else 1
    
    if returns is None:
        returns = prices.pct_change()
    
    if method == 'historical':
        # Simple rolling standard deviation
        vol = returns.rolling(window=window).std() * annualization_factor
        
    elif method == 'ewma':
        # Exponentially Weighted Moving Average
        vol = returns.ewm(span=window, adjust=False).std() * annualization_factor
        
    elif method == 'parkinson':
        # Parkinson volatility (uses high-low range)
        # Requires high/low prices in a DataFrame
        if isinstance(prices, pd.DataFrame) and 'high' in prices.columns and 'low' in prices.columns:
            log_hl = np.log(prices['high'] / prices['low'])
            vol = np.sqrt((log_hl ** 2).rolling(window=window).mean() / (4 * np.log(2))) * annualization_factor
        else:
            vol = pd.Series(np.nan, index=prices.index)
            
    elif method == 'garman_klass':
        # Garman-Klass volatility
        if isinstance(prices, pd.DataFrame) and all(c in prices.columns for c in ['high', 'low', 'open', 'close']):
            log_hl = np.log(prices['high'] / prices['low'])
            log_co = np.log(prices['close'] / prices['open'])
            gk = 0.5 * (log_hl ** 2) - (2 * np.log(2) - 1) * (log_co ** 2)
            vol = np.sqrt(gk.rolling(window=window).mean()) * annualization_factor
        else:
            vol = pd.Series(np.nan, index=prices.index)
            
    elif method == 'rogers_satchell':
        # Rogers-Satchell volatility
        if isinstance(prices, pd.DataFrame) and all(c in prices.columns for c in ['high', 'low', 'open', 'close']):
            log_hc = np.log(prices['high'] / prices['close'])
            log_ho = np.log(prices['high'] / prices['open'])
            log_lc = np.log(prices['low'] / prices['close'])
            log_lo = np.log(prices['low'] / prices['open'])
            rs = log_hc * log_ho + log_lc * log_lo
            vol = np.sqrt(rs.rolling(window=window).mean()) * annualization_factor
        else:
            vol = pd.Series(np.nan, index=prices.index)
            
    elif method == 'atr':
        # Average True Range based volatility
        if isinstance(prices, pd.DataFrame) and all(c in prices.columns for c in ['high', 'low', 'close']):
            prev_close = prices['close'].shift(1)
            tr = np.maximum(
                prices['high'] - prices['low'],
                np.maximum(
                    np.abs(prices['high'] - prev_close),
                    np.abs(prices['low'] - prev_close)
                )
            )
            atr = tr.rolling(window=window).mean()
            vol = (atr / prices['close']) * annualization_factor
        else:
            vol = pd.Series(np.nan, index=prices.index)
    else:
        vol = pd.Series(np.nan, index=prices.index if hasattr(prices, 'index') else None)
    
    return vol


# Calculate true realized volatility (multiple windows)
realized_vol = pd.DataFrame(index=df_asset.index)

for window in VOL_WINDOWS:
    realized_vol[f'rv_{window}d'] = calculate_volatility(
        prices=df_asset['close'],
        returns=df_asset['returns'],
        method='historical',
        window=window
    )

# Primary realized volatility (20-day is standard)
realized_vol['rv_true'] = realized_vol['rv_20d']

print("Realized Volatility Statistics:")
print(realized_vol.describe().round(4))

### 2.2 Multiple Volatility Models

In [ ]:
# ============================================================================
# MULTIPLE VOLATILITY MODELS
# ============================================================================

# Calculate all volatility measures
vol_models = pd.DataFrame(index=df_asset.index)

# 1. Historical volatility (various windows)
for window in VOL_WINDOWS:
    vol_models[f'hist_vol_{window}d'] = calculate_volatility(
        prices=df_asset['close'],
        returns=df_asset['returns'],
        method='historical',
        window=window
    )

# 2. EWMA volatility
for window in VOL_WINDOWS:
    vol_models[f'ewma_vol_{window}d'] = calculate_volatility(
        prices=df_asset['close'],
        returns=df_asset['returns'],
        method='ewma',
        window=window
    )

# 3. Parkinson volatility (high-low based)
for window in VOL_WINDOWS:
    vol_models[f'parkinson_{window}d'] = calculate_volatility(
        prices=df_asset,
        method='parkinson',
        window=window
    )

# 4. Garman-Klass volatility
for window in VOL_WINDOWS:
    vol_models[f'garman_klass_{window}d'] = calculate_volatility(
        prices=df_asset,
        method='garman_klass',
        window=window
    )

# 5. Rogers-Satchell volatility
for window in VOL_WINDOWS:
    vol_models[f'rogers_satchell_{window}d'] = calculate_volatility(
        prices=df_asset,
        method='rogers_satchell',
        window=window
    )

# 6. ATR-based volatility
for window in VOL_WINDOWS:
    vol_models[f'atr_vol_{window}d'] = calculate_volatility(
        prices=df_asset,
        method='atr',
        window=window
    )

# Drop NaN rows
vol_models = vol_models.dropna()

print(f"Volatility models calculated: {vol_models.shape[1]} features")
print(f"Valid observations: {len(vol_models)}")
print(f"\nVolatility Model Columns:")
for col in vol_models.columns:
    print(f"  - {col}")

### 2.3 Model Comparison Visualization

In [ ]:
# ============================================================================
# VOLATILITY MODEL COMPARISON
# ============================================================================

# Calculate correlation and RMSE for each model vs true volatility
true_vol = realized_vol['rv_true'].dropna()
common_idx = true_vol.index.intersection(vol_models.index)

model_metrics = []

for col in vol_models.columns:
    model_vol = vol_models[col].loc[common_idx]
    true_vol_aligned = true_vol.loc[common_idx]
    
    # Calculate metrics
    correlation = model_vol.corr(true_vol_aligned)
    rmse = np.sqrt(((model_vol - true_vol_aligned) ** 2).mean())
    mae = np.abs(model_vol - true_vol_aligned).mean()
    
    model_metrics.append({
        'Model': col,
        'Correlation': correlation,
        'RMSE': rmse,
        'MAE': mae
    })

metrics_df = pd.DataFrame(model_metrics).sort_values('Correlation', ascending=False)

print("Model Performance vs True Volatility (20d realized):")
print(metrics_df.to_string(index=False))

# Visualization: Model comparison
fig, axes = plt.subplots(2, 1, figsize=FIG_SIZE_COMPARISON)

# Top 5 models by correlation
top_models = metrics_df.head(5)['Model'].tolist()

# Plot 1: Time series comparison
ax1 = axes[0]
ax1.plot(true_vol.loc[common_idx], label='True Volatility (20d)', color='black', linewidth=2)
colors = plt.cm.tab10(np.linspace(0, 1, len(top_models)))
for i, model in enumerate(top_models):
    ax1.plot(vol_models[model].loc[common_idx], label=model, alpha=0.7, color=colors[i])
ax1.set_title(f'{TICKER} - Volatility Models Comparison', fontsize=14)
ax1.set_ylabel('Annualized Volatility')
ax1.legend(loc='upper right', fontsize=9)
ax1.grid(True, alpha=0.3)

# Plot 2: Correlation bar chart
ax2 = axes[1]
colors = ['#2ecc71' if x > 0.9 else '#f39c12' if x > 0.8 else '#e74c3c' 
          for x in metrics_df['Correlation'].head(10)]
ax2.barh(metrics_df['Model'].head(10)[::-1], metrics_df['Correlation'].head(10)[::-1], color=colors)
ax2.set_xlabel('Correlation with True Volatility')
ax2.set_title('Top 10 Models by Correlation', fontsize=14)
ax2.axvline(x=0.9, color='green', linestyle='--', label='High correlation (0.9)')
ax2.axvline(x=0.8, color='orange', linestyle='--', label='Good correlation (0.8)')
ax2.legend(loc='lower right', fontsize=9)

plt.tight_layout()
plt.show()

### 2.4 PCA-Based Aggregate Volatility Model

In [ ]:
# ============================================================================
# PCA-BASED AGGREGATE VOLATILITY MODEL
# ============================================================================

def run_pca_model(features: pd.DataFrame, n_components: int = 3, 
                  lag_periods: list = None) -> Tuple[pd.DataFrame, PCA, StandardScaler]:
    """
    Create PCA-based aggregate volatility forecast using lagged features.
    
    Parameters:
    -----------
    features : pd.DataFrame
        Volatility features
    n_components : int
        Number of PCA components
    lag_periods : list
        Lag periods to create
        
    Returns:
    --------
    Tuple[pd.DataFrame, PCA, StandardScaler]
        Lagged features with PCA components, fitted PCA model, fitted scaler
    """
    if lag_periods is None:
        lag_periods = [1, 2, 3, 5, 10, 20]
    
    # Create lagged features
    lagged_features = features.copy()
    
    for col in features.columns:
        for lag in lag_periods:
            lagged_features[f'{col}_lag{lag}'] = features[col].shift(lag)
    
    # Drop NaN rows
    lagged_features = lagged_features.dropna()
    
    # Standardize features
    scaler = StandardScaler()
    scaled_features = scaler.fit_transform(lagged_features)
    
    # Apply PCA
    pca = PCA(n_components=n_components)
    pca_components = pca.fit_transform(scaled_features)
    
    # Create output DataFrame
    result = pd.DataFrame(
        pca_components,
        index=lagged_features.index,
        columns=[f'PC{i+1}' for i in range(n_components)]
    )
    
    # Add explained variance info
    print(f"PCA Explained Variance Ratios:")
    for i, ratio in enumerate(pca.explained_variance_ratio_):
        print(f"  PC{i+1}: {ratio:.4f} ({ratio*100:.1f}%)")
    print(f"  Total: {pca.explained_variance_ratio_.sum():.4f} ({pca.explained_variance_ratio_.sum()*100:.1f}%)")
    
    return result, pca, scaler


# Run PCA on volatility features
pca_result, pca_model, vol_scaler = run_pca_model(
    features=vol_models,
    n_components=PCA_N_COMPONENTS,
    lag_periods=PCA_LAG_PERIODS
)

# Create aggregate volatility forecast (weighted combination of PCs)
# Weight by explained variance ratio
weights = pca_model.explained_variance_ratio_
pca_result['aggregate_vol_signal'] = sum(
    pca_result[f'PC{i+1}'] * weights[i] for i in range(PCA_N_COMPONENTS)
)

# Normalize to same scale as true volatility
true_vol_aligned = realized_vol['rv_true'].loc[pca_result.index]
signal_mean = pca_result['aggregate_vol_signal'].mean()
signal_std = pca_result['aggregate_vol_signal'].std()
true_mean = true_vol_aligned.mean()
true_std = true_vol_aligned.std()

pca_result['aggregate_vol_forecast'] = (
    (pca_result['aggregate_vol_signal'] - signal_mean) / signal_std * true_std + true_mean
)

print(f"\nAggregate Model Statistics:")
print(f"  Correlation with True Vol: {pca_result['aggregate_vol_forecast'].corr(true_vol_aligned):.4f}")
rmse = np.sqrt(((pca_result['aggregate_vol_forecast'] - true_vol_aligned) ** 2).mean())
print(f"  RMSE: {rmse:.4f}")

### 2.5 Aggregate Model Visualization

In [ ]:
# ============================================================================
# AGGREGATE MODEL VISUALIZATION
# ============================================================================

fig, axes = plt.subplots(3, 1, figsize=(14, 12))

# Plot 1: Aggregate forecast vs True volatility
ax1 = axes[0]
ax1.plot(true_vol_aligned, label='True Volatility (20d)', color='black', linewidth=2)
ax1.plot(pca_result['aggregate_vol_forecast'], label='PCA Aggregate Forecast', 
         color='#3498db', linewidth=1.5, alpha=0.8)
ax1.fill_between(pca_result.index, 
                 pca_result['aggregate_vol_forecast'] - rmse,
                 pca_result['aggregate_vol_forecast'] + rmse,
                 alpha=0.2, color='#3498db', label='±RMSE band')
ax1.set_title(f'{TICKER} - PCA Aggregate Volatility Model vs True Volatility', fontsize=14)
ax1.set_ylabel('Annualized Volatility')
ax1.legend(loc='upper right')
ax1.grid(True, alpha=0.3)

# Calculate correlation
corr = pca_result['aggregate_vol_forecast'].corr(true_vol_aligned)
ax1.annotate(f'Correlation: {corr:.4f}\nRMSE: {rmse:.4f}', 
             xy=(0.02, 0.98), xycoords='axes fraction',
             verticalalignment='top', fontsize=10,
             bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

# Plot 2: Principal Components
ax2 = axes[1]
for i in range(PCA_N_COMPONENTS):
    ax2.plot(pca_result[f'PC{i+1}'], label=f'PC{i+1} ({pca_model.explained_variance_ratio_[i]*100:.1f}%)',
             alpha=0.8)
ax2.set_title('Principal Components Over Time', fontsize=14)
ax2.set_ylabel('Standardized Value')
ax2.legend(loc='upper right')
ax2.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
ax2.grid(True, alpha=0.3)

# Plot 3: Scatter plot - Forecast vs Actual
ax3 = axes[2]
ax3.scatter(true_vol_aligned, pca_result['aggregate_vol_forecast'], alpha=0.5, s=10)
min_val = min(true_vol_aligned.min(), pca_result['aggregate_vol_forecast'].min())
max_val = max(true_vol_aligned.max(), pca_result['aggregate_vol_forecast'].max())
ax3.plot([min_val, max_val], [min_val, max_val], 'r--', linewidth=2, label='Perfect Forecast')
ax3.set_xlabel('True Volatility')
ax3.set_ylabel('Forecast Volatility')
ax3.set_title('Forecast vs Actual Volatility', fontsize=14)
ax3.legend(loc='upper left')
ax3.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---

## Section 3: Model Explanation & Maintenance

### 3.1 Understanding PCA (Principal Component Analysis)

**What is PCA?**

Principal Component Analysis is a dimensionality reduction technique that identifies patterns of correlation across multiple variables. In our volatility modeling context:

1. **Multiple Volatility Measures**: We calculate many volatility estimates (historical, EWMA, Parkinson, Garman-Klass, etc.) at different time windows. These measures are often highly correlated but capture slightly different aspects of volatility.

2. **Common Variance Patterns**: PCA identifies the common underlying "factors" that drive these volatility measures. The first principal component (PC1) captures the most common variance pattern - essentially what all volatility measures agree on.

3. **Noise Reduction**: By focusing on the top principal components (which explain most variance), we filter out measurement noise and idiosyncratic differences between methods.

4. **Composite Signal**: The weighted combination of principal components creates a more robust volatility forecast than any single measure.

**Interpretation of Principal Components:**
- **PC1** (largest variance): Usually represents the "level" of volatility - whether vol is generally high or low
- **PC2**: Often captures the "slope" - whether vol is rising or falling
- **PC3**: May capture "curvature" or regime-specific patterns

### 3.2 Why Lag Features Improve Volatility Forecasting

**Volatility Clustering**: One of the most robust findings in financial markets is that volatility exhibits strong autocorrelation - high volatility tends to be followed by high volatility, and low by low. This is captured by including lagged features.

**Optimal Lag Periods:**
- **Lag 1-3 days**: Captures immediate momentum and short-term persistence
- **Lag 5-10 days**: Captures weekly patterns and medium-term trends
- **Lag 20 days**: Captures monthly cycles and mean-reversion signals

**Why This Works:**
- Volatility at time t is highly correlated with volatility at t-1, t-2, etc.
- Changes in the relationship between current and lagged volatility signal regime changes
- The PCA captures how these lag relationships evolve over time

### 3.3 Model Maintenance Strategy

#### Rolling Window Retraining Schedule

The PCA model should be periodically retrained to adapt to changing market conditions:

| Frequency | Action | Rationale |
|-----------|--------|----------|
| Weekly | Monitor explained variance | Early warning of model drift |
| Monthly | Validate on recent data | Check out-of-sample performance |
| Quarterly | Full retraining | Adapt to structural changes |
| After major events | Emergency recalibration | Respond to regime shifts |

#### Out-of-Sample Validation Approach

```python
# Recommended validation structure:
# - Training: 70% of historical data
# - Validation: 15% for hyperparameter tuning
# - Test: 15% for final evaluation
# - Rolling: Move window forward and repeat
```

#### Drift Detection Methods

1. **Explained Variance Monitoring**: If total explained variance drops significantly, component structure may be changing
2. **Correlation Decay**: Track rolling correlation between forecast and realized volatility
3. **RMSE Tracking**: Monitor prediction error over rolling windows
4. **Distribution Shift**: Compare feature distributions using KS tests

#### Recalibration Triggers

| Metric | Threshold | Action |
|--------|-----------|--------|
| 30-day rolling correlation | < 0.7 | Alert & investigate |
| 30-day rolling RMSE | > 1.5× baseline | Trigger retraining |
| Explained variance (PC1-3) | < 70% | Review feature set |
| Consecutive days of underperformance | > 10 | Emergency review |

In [ ]:
# ============================================================================
# SECTION 3: MODEL MAINTENANCE UTILITIES
# ============================================================================

def calculate_model_drift_metrics(true_vol: pd.Series, forecast_vol: pd.Series,
                                  window: int = 30) -> pd.DataFrame:
    """
    Calculate rolling metrics to detect model drift.
    
    Parameters:
    -----------
    true_vol : pd.Series
        Realized volatility
    forecast_vol : pd.Series
        Model forecast
    window : int
        Rolling window for metrics
        
    Returns:
    --------
    pd.DataFrame
        Drift metrics over time
    """
    # Align series
    common_idx = true_vol.index.intersection(forecast_vol.index)
    true_vol = true_vol.loc[common_idx]
    forecast_vol = forecast_vol.loc[common_idx]
    
    drift_metrics = pd.DataFrame(index=common_idx)
    
    # Rolling correlation
    drift_metrics['rolling_corr'] = true_vol.rolling(window).corr(forecast_vol)
    
    # Rolling RMSE
    errors_sq = (true_vol - forecast_vol) ** 2
    drift_metrics['rolling_rmse'] = np.sqrt(errors_sq.rolling(window).mean())
    
    # Rolling MAE
    errors_abs = np.abs(true_vol - forecast_vol)
    drift_metrics['rolling_mae'] = errors_abs.rolling(window).mean()
    
    # Directional accuracy (did we predict direction of change correctly?)
    true_direction = np.sign(true_vol.diff())
    forecast_direction = np.sign(forecast_vol.diff())
    correct_direction = (true_direction == forecast_direction).astype(int)
    drift_metrics['directional_accuracy'] = correct_direction.rolling(window).mean()
    
    return drift_metrics.dropna()


def check_recalibration_needed(drift_metrics: pd.DataFrame, 
                               corr_threshold: float = 0.7,
                               rmse_multiplier: float = 1.5) -> dict:
    """
    Check if model needs recalibration based on drift metrics.
    
    Returns:
    --------
    dict
        Recalibration status and recommendations
    """
    latest = drift_metrics.iloc[-1]
    baseline_rmse = drift_metrics['rolling_rmse'].median()
    
    alerts = []
    
    if latest['rolling_corr'] < corr_threshold:
        alerts.append(f"Low correlation: {latest['rolling_corr']:.3f} < {corr_threshold}")
    
    if latest['rolling_rmse'] > baseline_rmse * rmse_multiplier:
        alerts.append(f"High RMSE: {latest['rolling_rmse']:.4f} > {baseline_rmse * rmse_multiplier:.4f}")
    
    if latest['directional_accuracy'] < 0.5:
        alerts.append(f"Poor directional accuracy: {latest['directional_accuracy']:.2%}")
    
    return {
        'needs_recalibration': len(alerts) > 0,
        'alerts': alerts,
        'latest_metrics': latest.to_dict()
    }


# Calculate drift metrics for our model
drift_metrics = calculate_model_drift_metrics(
    true_vol_aligned,
    pca_result['aggregate_vol_forecast'],
    window=30
)

# Check recalibration status
recal_status = check_recalibration_needed(drift_metrics)

print("Model Health Check:")
print(f"  Needs Recalibration: {recal_status['needs_recalibration']}")
if recal_status['alerts']:
    print("  Alerts:")
    for alert in recal_status['alerts']:
        print(f"    - {alert}")
print(f"\nLatest Metrics:")
for metric, value in recal_status['latest_metrics'].items():
    print(f"  {metric}: {value:.4f}")

---

## Section 4: Mean Reversion Analysis

This section analyzes volatility mean reversion characteristics to inform optimal options expiration selection.

### 4.1 Volatility Z-Score and Regime Classification

In [ ]:
# ============================================================================
# SECTION 4: MEAN REVERSION ANALYSIS
# ============================================================================

def calculate_volatility_zscore(vol_series: pd.Series, lookback: int = 252) -> pd.Series:
    """
    Calculate rolling z-score of volatility.
    
    Parameters:
    -----------
    vol_series : pd.Series
        Volatility series
    lookback : int
        Lookback window for mean and std calculation
        
    Returns:
    --------
    pd.Series
        Z-score series
    """
    rolling_mean = vol_series.rolling(window=lookback).mean()
    rolling_std = vol_series.rolling(window=lookback).std()
    zscore = (vol_series - rolling_mean) / rolling_std
    return zscore


def classify_volatility_regime(zscore: pd.Series) -> pd.Series:
    """
    Classify volatility into regimes based on z-score.
    
    Regimes:
    - Low: z < -1
    - Normal: -1 <= z <= 1
    - Elevated: 1 < z <= 2
    - Extreme: z > 2
    """
    conditions = [
        zscore < -1,
        (zscore >= -1) & (zscore <= 1),
        (zscore > 1) & (zscore <= 2),
        zscore > 2
    ]
    choices = ['Low', 'Normal', 'Elevated', 'Extreme']
    return pd.Series(np.select(conditions, choices, default='Normal'), index=zscore.index)


# Calculate z-scores
vol_zscore = calculate_volatility_zscore(realized_vol['rv_true'].dropna())
vol_regime = classify_volatility_regime(vol_zscore)

# Combine into analysis DataFrame
vol_analysis = pd.DataFrame({
    'volatility': realized_vol['rv_true'],
    'zscore': vol_zscore,
    'regime': vol_regime
}).dropna()

# Regime statistics
regime_stats = vol_analysis.groupby('regime').agg({
    'volatility': ['mean', 'std', 'count'],
    'zscore': ['mean', 'std']
}).round(4)

print("Volatility Regime Statistics:")
print(regime_stats)
print(f"\nRegime Distribution:")
regime_counts = vol_analysis['regime'].value_counts()
for regime, count in regime_counts.items():
    pct = count / len(vol_analysis) * 100
    print(f"  {regime}: {count} days ({pct:.1f}%)")

### 4.2 Mean Reversion Speed Analysis

In [ ]:
# ============================================================================
# MEAN REVERSION SPEED ANALYSIS
# ============================================================================

def calculate_mean_reversion_speed(zscore: pd.Series, regime_series: pd.Series) -> pd.DataFrame:
    """
    Calculate mean reversion characteristics for each volatility regime.
    Vectorized implementation.
    
    Returns:
    --------
    pd.DataFrame
        Mean reversion metrics by regime
    """
    results = []
    
    for regime in ['Low', 'Normal', 'Elevated', 'Extreme']:
        # Find regime entry points
        regime_mask = regime_series == regime
        regime_start = regime_mask & ~regime_mask.shift(1, fill_value=False)
        
        # For each entry, measure time to revert
        times_to_05 = []  # Time to reach ±0.5σ
        times_to_10 = []  # Time to reach ±1σ
        
        entry_indices = zscore.index[regime_start]
        
        for entry_idx in entry_indices:
            entry_zscore = zscore.loc[entry_idx]
            future_zscore = zscore.loc[entry_idx:]
            
            # Time to ±0.5σ
            mask_05 = np.abs(future_zscore) <= 0.5
            if mask_05.any():
                first_05 = mask_05.idxmax()
                days_to_05 = (first_05 - entry_idx).days
                if days_to_05 > 0:
                    times_to_05.append(days_to_05)
            
            # Time to ±1σ (only relevant for Elevated/Extreme)
            if regime in ['Elevated', 'Extreme']:
                mask_10 = np.abs(future_zscore) <= 1.0
                if mask_10.any():
                    first_10 = mask_10.idxmax()
                    days_to_10 = (first_10 - entry_idx).days
                    if days_to_10 > 0:
                        times_to_10.append(days_to_10)
        
        # Calculate half-life (approximate from exponential decay)
        if len(times_to_05) > 0:
            avg_time_05 = np.mean(times_to_05)
            median_time_05 = np.median(times_to_05)
        else:
            avg_time_05 = np.nan
            median_time_05 = np.nan
            
        if len(times_to_10) > 0:
            avg_time_10 = np.mean(times_to_10)
            median_time_10 = np.median(times_to_10)
        else:
            avg_time_10 = np.nan
            median_time_10 = np.nan
        
        results.append({
            'Regime': regime,
            'Entry_Count': len(entry_indices),
            'Avg_Days_to_0.5σ': avg_time_05,
            'Median_Days_to_0.5σ': median_time_05,
            'Avg_Days_to_1σ': avg_time_10,
            'Median_Days_to_1σ': median_time_10,
        })
    
    return pd.DataFrame(results)


# Calculate mean reversion metrics
reversion_metrics = calculate_mean_reversion_speed(vol_zscore.dropna(), vol_regime.dropna())

print("Mean Reversion Speed by Volatility Regime:")
print(reversion_metrics.to_string(index=False))

# Recommended DTE by regime
print("\n" + "="*60)
print("RECOMMENDED OPTIONS EXPIRATION BY VOLATILITY REGIME:")
print("="*60)

dte_recommendations = {
    'Extreme': '7-14 DTE (quick mean reversion expected)',
    'Elevated': '14-21 DTE (moderate reversion timeframe)',
    'Normal': '30-45 DTE (standard theta decay window)',
    'Low': '45-60 DTE (wait for volatility expansion)'
}

for regime, rec in dte_recommendations.items():
    print(f"  {regime}: {rec}")

### 4.3 Volatility Regime Transition Matrix (PRIORITY VISUALIZATION)

In [ ]:
# ============================================================================
# VOLATILITY REGIME TRANSITION MATRIX
# ============================================================================

def calculate_regime_transitions(regime_series: pd.Series) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """
    Calculate regime transition probability matrix.
    
    Returns:
    --------
    Tuple[pd.DataFrame, pd.DataFrame]
        Transition probability matrix and transition count matrix
    """
    regimes = ['Low', 'Normal', 'Elevated', 'Extreme']
    
    # Create transition count matrix
    transition_counts = pd.DataFrame(0, index=regimes, columns=regimes)
    
    current_regime = regime_series.iloc[:-1].values
    next_regime = regime_series.iloc[1:].values
    
    for curr, nxt in zip(current_regime, next_regime):
        if curr in regimes and nxt in regimes:
            transition_counts.loc[curr, nxt] += 1
    
    # Calculate probabilities
    row_sums = transition_counts.sum(axis=1)
    transition_probs = transition_counts.div(row_sums, axis=0)
    
    return transition_probs, transition_counts


def calculate_avg_regime_duration(regime_series: pd.Series) -> pd.DataFrame:
    """
    Calculate average duration spent in each regime.
    """
    regimes = ['Low', 'Normal', 'Elevated', 'Extreme']
    durations = {regime: [] for regime in regimes}
    
    current_regime = None
    current_duration = 0
    
    for regime in regime_series.values:
        if regime == current_regime:
            current_duration += 1
        else:
            if current_regime in durations:
                durations[current_regime].append(current_duration)
            current_regime = regime
            current_duration = 1
    
    # Add final regime
    if current_regime in durations:
        durations[current_regime].append(current_duration)
    
    # Calculate statistics
    duration_stats = []
    for regime in regimes:
        if durations[regime]:
            duration_stats.append({
                'Regime': regime,
                'Avg_Duration_Days': np.mean(durations[regime]),
                'Median_Duration_Days': np.median(durations[regime]),
                'Max_Duration_Days': np.max(durations[regime]),
                'Num_Episodes': len(durations[regime])
            })
    
    return pd.DataFrame(duration_stats)


# Calculate transitions
transition_probs, transition_counts = calculate_regime_transitions(vol_regime.dropna())
duration_stats = calculate_avg_regime_duration(vol_regime.dropna())

print("Regime Transition Probabilities:")
print(transition_probs.round(3))
print("\nAverage Regime Duration:")
print(duration_stats.to_string(index=False))

In [ ]:
# ============================================================================
# PRIORITY VISUALIZATION: Regime Transition Heatmap
# ============================================================================

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Transition probability heatmap
ax1 = axes[0]
sns.heatmap(transition_probs, annot=True, fmt='.2%', cmap='YlOrRd',
            ax=ax1, vmin=0, vmax=1, linewidths=0.5,
            cbar_kws={'label': 'Transition Probability'})
ax1.set_title('Volatility Regime Transition Probabilities\n(Row = Current Regime, Column = Next Regime)', 
              fontsize=14)
ax1.set_xlabel('Next Regime')
ax1.set_ylabel('Current Regime')

# Add average transition times as annotations
for i, regime_from in enumerate(transition_probs.index):
    duration = duration_stats[duration_stats['Regime'] == regime_from]['Avg_Duration_Days'].values
    if len(duration) > 0:
        ax1.annotate(f'Avg: {duration[0]:.1f}d', 
                    xy=(-0.3, i + 0.5), xycoords='data',
                    fontsize=9, ha='right', va='center')

# Bar chart of regime durations
ax2 = axes[1]
colors = [get_vol_regime_color(r) for r in duration_stats['Regime']]
bars = ax2.bar(duration_stats['Regime'], duration_stats['Avg_Duration_Days'], color=colors, alpha=0.8)
ax2.errorbar(duration_stats['Regime'], duration_stats['Avg_Duration_Days'],
             yerr=[duration_stats['Avg_Duration_Days'] - duration_stats['Median_Duration_Days'],
                   duration_stats['Max_Duration_Days'] - duration_stats['Avg_Duration_Days']],
             fmt='none', color='black', capsize=5)
ax2.set_title('Average Regime Duration (with min/max range)', fontsize=14)
ax2.set_xlabel('Volatility Regime')
ax2.set_ylabel('Days')
ax2.grid(True, alpha=0.3, axis='y')

# Add count annotations
for bar, count in zip(bars, duration_stats['Num_Episodes']):
    ax2.annotate(f'n={count}', 
                xy=(bar.get_x() + bar.get_width()/2, bar.get_height()),
                xytext=(0, 5), textcoords='offset points',
                ha='center', va='bottom', fontsize=10)

plt.tight_layout()
plt.show()

print("\nKEY INSIGHTS:")
print("- High diagonal values indicate regime persistence")
print("- Off-diagonal values show transition likelihoods")
print("- Extreme volatility typically reverts to Elevated/Normal")
print("- Use this to inform position sizing and DTE selection")

---

## Section 5: Options Strategy Development & Delta Analysis

This section implements six options strategies with systematic delta testing and performance analysis.

### Strategy Suite:
1. **Calendar Spread** - Long back month, short front month, same strike
2. **Double Diagonal** - Calendar spread on both calls and puts, different strikes
3. **Long Straddle** - Long ATM call + long ATM put
4. **Long Strangle** - Long OTM call + long OTM put
5. **Bull Put Spread** - Short put + long lower strike put
6. **Bull Call Spread** - Long call + short higher strike call

### 5.1 Strategy Implementation Functions

In [ ]:
# ============================================================================
# SECTION 5: OPTIONS STRATEGY DEVELOPMENT
# ============================================================================

# Strategy configurations
STRATEGIES = [
    'Calendar Spread',
    'Double Diagonal',
    'Straddle',
    'Strangle',
    'Bull Put Spread',
    'Bull Call Spread'
]

def calculate_strategy_pnl(strategy_type: str, entry_price: float, exit_price: float,
                           contracts: int = 1, multiplier: int = 100) -> float:
    """
    Calculate P&L for a strategy.
    
    Parameters:
    -----------
    strategy_type : str
        Type of options strategy
    entry_price : float
        Net debit/credit at entry
    exit_price : float
        Net value at exit
    contracts : int
        Number of contracts
    multiplier : int
        Contract multiplier (usually 100)
        
    Returns:
    --------
    float
        Profit/Loss in dollars
    """
    return (exit_price - entry_price) * contracts * multiplier


def apply_stop_loss(pnl_series: pd.Series, stop_threshold: float = None) -> pd.Series:
    """
    Apply stop loss to a P&L series.
    Vectorized implementation.
    
    Parameters:
    -----------
    pnl_series : pd.Series
        Series of P&L values (as returns or dollar amounts)
    stop_threshold : float
        Stop loss threshold (e.g., -0.20 for 20% loss). None = no stop.
        
    Returns:
    --------
    pd.Series
        Adjusted P&L series with stop loss applied
    """
    if stop_threshold is None:
        return pnl_series
    
    # Apply stop loss - cap losses at threshold
    adjusted = pnl_series.copy()
    adjusted = np.where(adjusted < stop_threshold, stop_threshold, adjusted)
    return pd.Series(adjusted, index=pnl_series.index)


def roll_position(current_expiry: datetime, days_to_roll: int = 5) -> datetime:
    """
    Calculate next expiration date for position rolling.
    
    Parameters:
    -----------
    current_expiry : datetime
        Current position expiration
    days_to_roll : int
        Days before expiry to trigger roll
        
    Returns:
    --------
    datetime
        Next expiration date (approximately 30 days out)
    """
    # Roll to next monthly expiration
    next_expiry = current_expiry + timedelta(days=30)
    # Adjust to third Friday (standard monthly expiration)
    # Find first day of expiry month
    first_day = next_expiry.replace(day=1)
    # Find first Friday
    days_until_friday = (4 - first_day.weekday()) % 7
    first_friday = first_day + timedelta(days=days_until_friday)
    # Third Friday
    third_friday = first_friday + timedelta(days=14)
    return third_friday


print("Strategy functions defined.")
print(f"Strategies: {STRATEGIES}")

### 5.2 Simulated Strategy Returns

Since actual options data may not be available for all historical dates, we simulate strategy returns based on:
- Underlying price movements
- Volatility regime
- Strategy-specific sensitivities (delta, vega, theta)

In [ ]:
# ============================================================================
# SIMULATED STRATEGY RETURNS BASED ON UNDERLYING DYNAMICS
# ============================================================================

def simulate_strategy_returns(underlying_returns: pd.Series, volatility: pd.Series,
                              vol_regime: pd.Series, strategy: str,
                              delta_config: dict = None) -> pd.Series:
    """
    Simulate strategy returns based on underlying dynamics.
    Vectorized implementation.
    
    This is a simplified model - actual returns would use full options pricing.
    
    Parameters:
    -----------
    underlying_returns : pd.Series
        Daily returns of underlying
    volatility : pd.Series
        Realized volatility
    vol_regime : pd.Series
        Volatility regime classification
    strategy : str
        Strategy type
    delta_config : dict
        Delta configuration for the strategy
        
    Returns:
    --------
    pd.Series
        Simulated strategy returns
    """
    # Align all series
    common_idx = underlying_returns.dropna().index
    common_idx = common_idx.intersection(volatility.dropna().index)
    
    ret = underlying_returns.loc[common_idx]
    vol = volatility.loc[common_idx]
    
    # Normalized volatility for scaling
    vol_normalized = (vol - vol.mean()) / vol.std()
    
    # Strategy-specific return simulation
    if strategy == 'Calendar Spread':
        # Calendar spreads profit from stable prices and vol crush
        # Positive vega exposure, minimal delta
        theta_component = 0.001  # Daily theta decay benefit
        vega_component = -vol_normalized.diff() * 0.02  # Profit from vol decrease
        gamma_component = -np.abs(ret) * 0.5  # Loss from large moves
        strat_returns = theta_component + vega_component + gamma_component
        
    elif strategy == 'Double Diagonal':
        # Similar to calendar but wider profit zone
        theta_component = 0.0008
        vega_component = -vol_normalized.diff() * 0.015
        gamma_component = -np.abs(ret) * 0.3
        strat_returns = theta_component + vega_component + gamma_component
        
    elif strategy == 'Straddle':
        # Long straddle profits from large moves, loses theta
        theta_component = -0.002  # Daily theta decay cost
        gamma_component = np.abs(ret) * 2.0  # Profit from large moves
        vega_component = vol_normalized.diff() * 0.03  # Profit from vol increase
        strat_returns = theta_component + gamma_component + vega_component
        
    elif strategy == 'Strangle':
        # Similar to straddle but needs larger moves
        theta_component = -0.0015
        gamma_component = np.where(np.abs(ret) > 0.02, np.abs(ret) * 1.5, -0.001)
        vega_component = vol_normalized.diff() * 0.025
        strat_returns = theta_component + gamma_component + vega_component
        
    elif strategy == 'Bull Put Spread':
        # Credit spread - profits from stable/rising prices
        # Short delta exposure
        delta = delta_config.get('put_delta', -0.20) if delta_config else -0.20
        delta_component = ret * np.abs(delta) * 0.5
        theta_component = 0.0015
        gamma_component = np.where(ret < -0.02, ret * 2, 0)  # Loss from large drops
        strat_returns = delta_component + theta_component + gamma_component
        
    elif strategy == 'Bull Call Spread':
        # Debit spread - profits from rising prices
        delta = delta_config.get('call_delta', 0.40) if delta_config else 0.40
        delta_component = ret * delta * 0.8
        theta_component = -0.0008
        strat_returns = delta_component + theta_component
        
    else:
        strat_returns = pd.Series(0, index=common_idx)
    
    # Add noise for realism
    noise = np.random.normal(0, 0.002, len(strat_returns))
    strat_returns = strat_returns + noise
    
    return pd.Series(strat_returns, index=common_idx)


# Generate returns for all strategies
strategy_returns = {}

for strategy in STRATEGIES:
    returns = simulate_strategy_returns(
        underlying_returns=df_asset['returns'],
        volatility=realized_vol['rv_true'],
        vol_regime=vol_regime,
        strategy=strategy
    )
    strategy_returns[strategy] = returns
    
# Create DataFrame
strategy_returns_df = pd.DataFrame(strategy_returns)

print(f"Strategy returns generated: {len(strategy_returns_df)} days")
print("\nStrategy Return Statistics:")
print(strategy_returns_df.describe().round(4))

### 5.3 Delta Configuration Analysis

In [ ]:
# ============================================================================
# DELTA CONFIGURATION ANALYSIS
# ============================================================================

def backtest_strategy(strategy_type: str, delta_config: dict, 
                      underlying_returns: pd.Series, volatility: pd.Series,
                      vol_regime: pd.Series, stop_loss: float = None) -> dict:
    """
    Backtest a strategy with specific delta configuration.
    
    Returns:
    --------
    dict
        Performance metrics
    """
    # Generate returns
    returns = simulate_strategy_returns(
        underlying_returns=underlying_returns,
        volatility=volatility,
        vol_regime=vol_regime,
        strategy=strategy_type,
        delta_config=delta_config
    )
    
    # Apply stop loss
    returns = apply_stop_loss(returns, stop_loss)
    
    # Calculate metrics
    total_return = (1 + returns).prod() - 1
    annual_return = (1 + total_return) ** (252 / len(returns)) - 1
    volatility_ann = returns.std() * np.sqrt(252)
    sharpe = annual_return / volatility_ann if volatility_ann > 0 else 0
    
    # Drawdown
    cumulative = (1 + returns).cumprod()
    running_max = cumulative.expanding().max()
    drawdown = (cumulative - running_max) / running_max
    max_drawdown = drawdown.min()
    
    # Win rate
    win_rate = (returns > 0).mean()
    
    return {
        'strategy': strategy_type,
        'delta_config': str(delta_config),
        'stop_loss': stop_loss,
        'total_return': total_return,
        'annual_return': annual_return,
        'volatility': volatility_ann,
        'sharpe': sharpe,
        'max_drawdown': max_drawdown,
        'win_rate': win_rate,
        'n_days': len(returns)
    }


# Test different delta configurations for each strategy
delta_test_results = []

# Calendar Spread deltas
for delta in CALENDAR_DELTAS:
    config = {'call_delta': delta}
    result = backtest_strategy('Calendar Spread', config, 
                               df_asset['returns'], realized_vol['rv_true'], vol_regime)
    delta_test_results.append(result)

# Strangle delta pairs
for call_d, put_d in STRANGLE_DELTA_PAIRS:
    config = {'call_delta': call_d, 'put_delta': put_d}
    result = backtest_strategy('Strangle', config,
                               df_asset['returns'], realized_vol['rv_true'], vol_regime)
    delta_test_results.append(result)

# Bull Put Spread deltas
for put_d in BULL_PUT_SHORT_DELTAS:
    config = {'put_delta': put_d}
    result = backtest_strategy('Bull Put Spread', config,
                               df_asset['returns'], realized_vol['rv_true'], vol_regime)
    delta_test_results.append(result)

# Bull Call Spread deltas
for call_d in BULL_CALL_LONG_DELTAS:
    config = {'call_delta': call_d}
    result = backtest_strategy('Bull Call Spread', config,
                               df_asset['returns'], realized_vol['rv_true'], vol_regime)
    delta_test_results.append(result)

delta_results_df = pd.DataFrame(delta_test_results)

print("Delta Configuration Test Results:")
print(delta_results_df[['strategy', 'delta_config', 'sharpe', 'total_return', 'max_drawdown', 'win_rate']].to_string(index=False))

### 5.4 PRIORITY VISUALIZATION #1: Strategy Equity Curves Comparison

In [ ]:
# ============================================================================
# PRIORITY VISUALIZATION #1: Strategy Equity Curves
# ============================================================================

# Calculate cumulative returns for each strategy
cumulative_returns = (1 + strategy_returns_df).cumprod()

# Calculate buy-and-hold benchmark
benchmark_returns = df_asset['returns'].loc[strategy_returns_df.index]
benchmark_cumulative = (1 + benchmark_returns.fillna(0)).cumprod()

# Create figure
fig, ax = plt.subplots(figsize=(14, 8))

# Plot each strategy with consistent colors
for strategy in STRATEGIES:
    color = get_strategy_color(strategy)
    ax.plot(cumulative_returns[strategy], label=strategy, color=color, linewidth=1.5)

# Plot benchmark
ax.plot(benchmark_cumulative, label='Buy & Hold', color='gray', 
        linestyle='--', linewidth=2, alpha=0.8)

# Add regime shading
if len(vol_regime.dropna()) > 0:
    regime_aligned = vol_regime.reindex(cumulative_returns.index, method='ffill')
    
    # Shade by regime
    prev_regime = None
    regime_start = cumulative_returns.index[0]
    
    for idx, regime in regime_aligned.items():
        if regime != prev_regime and prev_regime is not None:
            color = get_vol_regime_color(prev_regime)
            ax.axvspan(regime_start, idx, alpha=0.1, color=color)
            regime_start = idx
        prev_regime = regime
    
    # Final regime
    if prev_regime:
        color = get_vol_regime_color(prev_regime)
        ax.axvspan(regime_start, cumulative_returns.index[-1], alpha=0.1, color=color)

ax.set_title(f'{TICKER} - Strategy Equity Curves Comparison', fontsize=14)
ax.set_xlabel('Date')
ax.set_ylabel('Cumulative Return (1 = Starting Value)')
ax.legend(loc='upper left', fontsize=10)
ax.grid(True, alpha=0.3)
ax.axhline(y=1, color='black', linestyle='-', linewidth=0.5)

# Add regime legend
regime_patches = [mpatches.Patch(color=get_vol_regime_color(r), alpha=0.3, label=r) 
                  for r in ['Low', 'Normal', 'Elevated', 'Extreme']]
ax.legend(handles=ax.get_legend_handles_labels()[0] + regime_patches, 
          loc='upper left', fontsize=9, ncol=2)

plt.tight_layout()
plt.show()

# Print final returns
print("\nFinal Cumulative Returns:")
for strategy in STRATEGIES:
    final_return = cumulative_returns[strategy].iloc[-1] - 1
    print(f"  {strategy}: {final_return:.2%}")
print(f"  Buy & Hold: {benchmark_cumulative.iloc[-1] - 1:.2%}")

### 5.5 PRIORITY VISUALIZATION #2: Rolling Sharpe Ratio (Individual Plots)

In [ ]:
# ============================================================================
# PRIORITY VISUALIZATION #2: Rolling Sharpe Ratio
# ============================================================================

def calculate_rolling_sharpe(returns: pd.Series, window: int = 60) -> pd.Series:
    """Calculate rolling Sharpe ratio."""
    rolling_mean = returns.rolling(window=window).mean() * 252
    rolling_std = returns.rolling(window=window).std() * np.sqrt(252)
    return rolling_mean / rolling_std

# Create separate plots for each strategy
for strategy in STRATEGIES:
    fig, ax = plt.subplots(figsize=(12, 5))
    
    color = get_strategy_color(strategy)
    
    # Calculate rolling Sharpe
    rolling_sharpe = calculate_rolling_sharpe(strategy_returns_df[strategy], ROLLING_SHARPE_WINDOW)
    
    # Plot smoothed line
    smoothed = rolling_sharpe.rolling(10).mean()
    ax.plot(smoothed, color=color, linewidth=2, label=f'{strategy} (60-day rolling)')
    
    # Add confidence bands (±1 std)
    rolling_std = rolling_sharpe.rolling(20).std()
    ax.fill_between(smoothed.index, 
                    smoothed - rolling_std, 
                    smoothed + rolling_std,
                    alpha=0.2, color=color)
    
    # Add regime background shading
    regime_aligned = vol_regime.reindex(rolling_sharpe.index, method='ffill')
    
    prev_regime = None
    regime_start = rolling_sharpe.dropna().index[0] if len(rolling_sharpe.dropna()) > 0 else None
    
    if regime_start is not None:
        for idx in rolling_sharpe.dropna().index:
            regime = regime_aligned.get(idx)
            if regime != prev_regime and prev_regime is not None:
                regime_color = get_vol_regime_color(prev_regime)
                ax.axvspan(regime_start, idx, alpha=0.15, color=regime_color)
                regime_start = idx
            prev_regime = regime
        
        # Final segment
        if prev_regime:
            regime_color = get_vol_regime_color(prev_regime)
            ax.axvspan(regime_start, rolling_sharpe.dropna().index[-1], alpha=0.15, color=regime_color)
    
    ax.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
    ax.axhline(y=1, color='green', linestyle='--', linewidth=0.5, alpha=0.5, label='Sharpe = 1')
    ax.axhline(y=-1, color='red', linestyle='--', linewidth=0.5, alpha=0.5, label='Sharpe = -1')
    
    ax.set_title(f'{strategy} - Rolling Sharpe Ratio ({ROLLING_SHARPE_WINDOW}-day window)', 
                 fontsize=14, color=color)
    ax.set_xlabel('Date')
    ax.set_ylabel('Sharpe Ratio')
    ax.legend(loc='upper right', fontsize=9)
    ax.grid(True, alpha=0.3)
    
    # Add overall Sharpe annotation
    overall_sharpe = (strategy_returns_df[strategy].mean() * 252) / (strategy_returns_df[strategy].std() * np.sqrt(252))
    ax.annotate(f'Overall Sharpe: {overall_sharpe:.2f}', 
                xy=(0.02, 0.98), xycoords='axes fraction',
                verticalalignment='top', fontsize=10,
                bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    
    plt.tight_layout()
    plt.show()
    print()

### 5.6 Stop Loss Optimization

In [ ]:
# ============================================================================
# STOP LOSS OPTIMIZATION
# ============================================================================

def optimize_strategy_parameters(strategy: str, underlying_returns: pd.Series,
                                 volatility: pd.Series, vol_regime: pd.Series,
                                 stop_loss_thresholds: list) -> pd.DataFrame:
    """
    Test multiple stop loss thresholds for a strategy.
    Vectorized implementation.
    """
    results = []
    
    # Get base returns
    base_returns = simulate_strategy_returns(
        underlying_returns=underlying_returns,
        volatility=volatility,
        vol_regime=vol_regime,
        strategy=strategy
    )
    
    for stop in stop_loss_thresholds:
        # Apply stop loss
        adjusted_returns = apply_stop_loss(base_returns, stop)
        
        # Calculate metrics
        total_return = (1 + adjusted_returns).prod() - 1
        annual_return = (1 + total_return) ** (252 / len(adjusted_returns)) - 1 if len(adjusted_returns) > 0 else 0
        vol_ann = adjusted_returns.std() * np.sqrt(252)
        sharpe = annual_return / vol_ann if vol_ann > 0 else 0
        
        cumulative = (1 + adjusted_returns).cumprod()
        running_max = cumulative.expanding().max()
        drawdown = (cumulative - running_max) / running_max
        max_dd = drawdown.min()
        
        win_rate = (adjusted_returns > 0).mean()
        
        # Count stopped trades
        if stop is not None:
            n_stopped = (base_returns < stop).sum()
        else:
            n_stopped = 0
        
        results.append({
            'strategy': strategy,
            'stop_loss': f'{stop:.0%}' if stop else 'None',
            'stop_value': stop if stop else 0,
            'total_return': total_return,
            'sharpe': sharpe,
            'max_drawdown': max_dd,
            'win_rate': win_rate,
            'n_stopped': n_stopped
        })
    
    return pd.DataFrame(results)


# Run stop loss optimization for each strategy
stop_loss_results = []

for strategy in STRATEGIES:
    results = optimize_strategy_parameters(
        strategy=strategy,
        underlying_returns=df_asset['returns'],
        volatility=realized_vol['rv_true'],
        vol_regime=vol_regime,
        stop_loss_thresholds=STOP_LOSS_THRESHOLDS
    )
    stop_loss_results.append(results)

stop_loss_df = pd.concat(stop_loss_results, ignore_index=True)

print("Stop Loss Optimization Results:")
print(stop_loss_df[['strategy', 'stop_loss', 'total_return', 'sharpe', 'max_drawdown', 'win_rate']].to_string(index=False))

In [ ]:
# ============================================================================
# STOP LOSS VISUALIZATION - Separate plots per strategy
# ============================================================================

for strategy in STRATEGIES:
    strategy_data = stop_loss_df[stop_loss_df['strategy'] == strategy]
    color = get_strategy_color(strategy)
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Left: Max Drawdown vs Total Return scatter
    ax1 = axes[0]
    scatter = ax1.scatter(
        -strategy_data['max_drawdown'] * 100,  # Convert to positive %
        strategy_data['total_return'] * 100,
        c=range(len(strategy_data)),
        cmap='viridis',
        s=100,
        edgecolors='black'
    )
    
    # Annotate points
    for idx, row in strategy_data.iterrows():
        ax1.annotate(row['stop_loss'],
                    xy=(-row['max_drawdown']*100, row['total_return']*100),
                    xytext=(5, 5), textcoords='offset points',
                    fontsize=8)
    
    ax1.set_xlabel('Max Drawdown (%)')
    ax1.set_ylabel('Total Return (%)')
    ax1.set_title(f'{strategy} - Return vs Drawdown by Stop Loss', fontsize=12, color=color)
    ax1.grid(True, alpha=0.3)
    
    # Right: Bar chart of Sharpe by stop loss
    ax2 = axes[1]
    bars = ax2.bar(strategy_data['stop_loss'], strategy_data['sharpe'], color=color, alpha=0.7)
    ax2.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
    ax2.set_xlabel('Stop Loss Threshold')
    ax2.set_ylabel('Sharpe Ratio')
    ax2.set_title(f'{strategy} - Sharpe Ratio by Stop Loss', fontsize=12, color=color)
    ax2.grid(True, alpha=0.3, axis='y')
    plt.xticks(rotation=45)
    
    plt.tight_layout()
    plt.show()
    
    # Identify optimal stop loss
    best_row = strategy_data.loc[strategy_data['sharpe'].idxmax()]
    print(f"{strategy} - Optimal Stop Loss: {best_row['stop_loss']} (Sharpe: {best_row['sharpe']:.2f})")
    print()

---

## Section 6: Market Regime Analysis Using Ensemble Clustering

This section implements regime detection using an ensemble of:
- **Gaussian Mixture Models (GMM)** - Probabilistic clustering
- **K-Means Clustering** - Hard cluster assignments
- **Agglomerative Clustering** - Hierarchical clustering
- **Change-Point Detection (CPD)** - Structural break detection

### Why Ensemble Approach?

| Method | Strengths | Weaknesses |
|--------|-----------|------------|
| GMM | Probability distributions, overlapping clusters | Sensitive to initialization |
| K-Means | Fast, simple, spherical clusters | Assumes equal cluster sizes |
| Agglomerative | Hierarchical structure, no shape assumption | Computationally intensive |
| CPD | Explicit transition detection | May miss gradual changes |

The ensemble combines these methods to reduce individual weaknesses and provide confidence scores.

### 6.1 Feature Engineering for Regime Detection

In [ ]:
# ============================================================================
# SECTION 6: ENSEMBLE REGIME DETECTION
# ============================================================================

def engineer_regime_features(data: pd.DataFrame, feature_set: str = 'comprehensive') -> pd.DataFrame:
    """
    Create comprehensive feature set for regime detection.
    Vectorized implementation.
    
    Parameters:
    -----------
    data : pd.DataFrame
        OHLCV data with 'close', 'high', 'low', 'volume' columns
    feature_set : str
        'comprehensive', 'returns_only', 'volatility_only'
        
    Returns:
    --------
    pd.DataFrame
        Feature DataFrame
    """
    features = pd.DataFrame(index=data.index)
    
    # Ensure we have returns
    if 'returns' not in data.columns:
        data = data.copy()
        data['returns'] = data['close'].pct_change()
    
    # 1. Return-based features
    features['returns_1d'] = data['returns']
    features['returns_5d'] = data['close'].pct_change(5)
    features['returns_10d'] = data['close'].pct_change(10)
    features['returns_20d'] = data['close'].pct_change(20)
    features['returns_50d'] = data['close'].pct_change(50)
    
    # Return volatility
    features['vol_5d'] = data['returns'].rolling(5).std()
    features['vol_10d'] = data['returns'].rolling(10).std()
    features['vol_20d'] = data['returns'].rolling(20).std()
    
    # Skewness and kurtosis
    features['skew_20d'] = data['returns'].rolling(20).skew()
    features['kurt_20d'] = data['returns'].rolling(20).kurt()
    
    if feature_set in ['comprehensive', 'volatility_only']:
        # 2. Volatility features
        features['vol_of_vol'] = features['vol_20d'].rolling(20).std()
        
        if 'high' in data.columns and 'low' in data.columns:
            features['hl_range'] = (data['high'] - data['low']) / data['close']
            features['hl_range_20d'] = features['hl_range'].rolling(20).mean()
    
    if feature_set == 'comprehensive':
        # 3. Trend features
        features['sma_20'] = data['close'].rolling(20).mean()
        features['sma_50'] = data['close'].rolling(50).mean()
        features['sma_200'] = data['close'].rolling(200).mean()
        
        features['price_sma20_ratio'] = data['close'] / features['sma_20']
        features['price_sma50_ratio'] = data['close'] / features['sma_50']
        features['sma20_sma50_ratio'] = features['sma_20'] / features['sma_50']
        
        # Linear regression slope (trend strength)
        def calc_slope(series, window=20):
            def slope(x):
                if len(x) < window:
                    return np.nan
                y = x.values
                x_vals = np.arange(len(y))
                slope, _ = np.polyfit(x_vals, y, 1)
                return slope
            return series.rolling(window).apply(slope, raw=False)
        
        features['trend_slope_20d'] = calc_slope(data['close'], 20)
        
        # 4. Momentum features
        # RSI
        delta = data['close'].diff()
        gain = delta.where(delta > 0, 0).rolling(14).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(14).mean()
        rs = gain / loss
        features['rsi_14'] = 100 - (100 / (1 + rs))
        
        # MACD
        ema_12 = data['close'].ewm(span=12, adjust=False).mean()
        ema_26 = data['close'].ewm(span=26, adjust=False).mean()
        features['macd'] = ema_12 - ema_26
        features['macd_signal'] = features['macd'].ewm(span=9, adjust=False).mean()
        features['macd_hist'] = features['macd'] - features['macd_signal']
        
        # Rate of change
        features['roc_10'] = data['close'].pct_change(10)
        features['roc_20'] = data['close'].pct_change(20)
        
        # 5. Volume features (if available)
        if 'volume' in data.columns:
            features['volume_sma_20'] = data['volume'].rolling(20).mean()
            features['volume_ratio'] = data['volume'] / features['volume_sma_20']
    
    # Drop columns used for intermediate calculations
    cols_to_keep = [c for c in features.columns if not c.startswith('sma_')]
    features = features[cols_to_keep]
    
    return features.dropna()


# Engineer features for market, sector, and asset
print("Engineering features...")

market_features = engineer_regime_features(df_market, 'comprehensive')
print(f"Market features: {market_features.shape}")

sector_features = engineer_regime_features(df_sector, 'comprehensive')
print(f"Sector features: {sector_features.shape}")

asset_features = engineer_regime_features(df_asset, 'comprehensive')
print(f"Asset features: {asset_features.shape}")

print(f"\nFeature columns ({len(market_features.columns)}):")
for col in market_features.columns:
    print(f"  - {col}")

### 6.2 Individual Clustering Methods

In [ ]:
# ============================================================================
# CLUSTERING METHODS
# ============================================================================

def fit_gmm_regimes(features: pd.DataFrame, n_components_range: range = range(3, 7)) -> Tuple[np.ndarray, GaussianMixture, int]:
    """
    Fit Gaussian Mixture Model for regime detection.
    Uses BIC for optimal component selection.
    
    Returns:
    --------
    Tuple[np.ndarray, GaussianMixture, int]
        Labels, fitted model, optimal n_components
    """
    # Standardize features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(features)
    
    # Find optimal n_components using BIC
    bic_scores = []
    models = []
    
    for n in n_components_range:
        gmm = GaussianMixture(n_components=n, random_state=42, n_init=5)
        gmm.fit(X_scaled)
        bic_scores.append(gmm.bic(X_scaled))
        models.append(gmm)
    
    # Select model with lowest BIC
    best_idx = np.argmin(bic_scores)
    best_n = list(n_components_range)[best_idx]
    best_model = models[best_idx]
    
    # Get labels and probabilities
    labels = best_model.predict(X_scaled)
    probabilities = best_model.predict_proba(X_scaled)
    
    print(f"GMM: Optimal components = {best_n} (BIC = {bic_scores[best_idx]:.2f})")
    
    return labels, best_model, best_n, probabilities


def fit_kmeans_regimes(features: pd.DataFrame, k_range: range = range(3, 7)) -> Tuple[np.ndarray, KMeans, int]:
    """
    Fit K-Means clustering for regime detection.
    Uses silhouette score for optimal k selection.
    """
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(features)
    
    silhouette_scores = []
    models = []
    
    for k in k_range:
        kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
        labels = kmeans.fit_predict(X_scaled)
        score = silhouette_score(X_scaled, labels)
        silhouette_scores.append(score)
        models.append(kmeans)
    
    best_idx = np.argmax(silhouette_scores)
    best_k = list(k_range)[best_idx]
    best_model = models[best_idx]
    labels = best_model.predict(X_scaled)
    
    # Calculate distance to cluster centers
    distances = best_model.transform(X_scaled)
    min_distances = distances.min(axis=1)
    
    print(f"K-Means: Optimal k = {best_k} (Silhouette = {silhouette_scores[best_idx]:.3f})")
    
    return labels, best_model, best_k, min_distances


def fit_agglomerative_regimes(features: pd.DataFrame, n_clusters_range: range = range(3, 7)) -> Tuple[np.ndarray, int]:
    """
    Fit Agglomerative (Hierarchical) clustering.
    """
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(features)
    
    silhouette_scores = []
    all_labels = []
    
    for n in n_clusters_range:
        agg = AgglomerativeClustering(n_clusters=n, linkage='ward')
        labels = agg.fit_predict(X_scaled)
        score = silhouette_score(X_scaled, labels)
        silhouette_scores.append(score)
        all_labels.append(labels)
    
    best_idx = np.argmax(silhouette_scores)
    best_n = list(n_clusters_range)[best_idx]
    best_labels = all_labels[best_idx]
    
    print(f"Agglomerative: Optimal clusters = {best_n} (Silhouette = {silhouette_scores[best_idx]:.3f})")
    
    return best_labels, best_n


def detect_change_points(features: pd.DataFrame, method: str = 'pelt', 
                         n_bkps: int = None) -> np.ndarray:
    """
    Detect structural breaks using change-point detection.
    
    Returns:
    --------
    np.ndarray
        Regime labels based on detected segments
    """
    if not RUPTURES_AVAILABLE:
        print("Warning: ruptures not available, returning uniform segments")
        return np.zeros(len(features))
    
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(features)
    
    if method == 'pelt':
        algo = rpt.Pelt(model='rbf').fit(X_scaled)
        result = algo.predict(pen=3)
    elif method == 'binseg':
        algo = rpt.Binseg(model='rbf').fit(X_scaled)
        result = algo.predict(n_bkps=n_bkps or 10)
    else:
        algo = rpt.BottomUp(model='rbf').fit(X_scaled)
        result = algo.predict(n_bkps=n_bkps or 10)
    
    # Convert change points to labels
    labels = np.zeros(len(features))
    prev_bkp = 0
    for i, bkp in enumerate(result):
        labels[prev_bkp:bkp] = i
        prev_bkp = bkp
    
    print(f"CPD ({method}): {len(result)} change points detected")
    
    return labels, result


print("Clustering functions defined.")

### 6.3 Ensemble Regime Classification

In [ ]:
# ============================================================================
# ENSEMBLE CLASSIFICATION
# ============================================================================

def ensemble_regime_classification(gmm_labels: np.ndarray, kmeans_labels: np.ndarray,
                                   agg_labels: np.ndarray, cpd_labels: np.ndarray,
                                   gmm_probs: np.ndarray = None,
                                   kmeans_distances: np.ndarray = None,
                                   weights: dict = None) -> Tuple[np.ndarray, np.ndarray]:
    """
    Combine multiple clustering methods using weighted voting.
    
    Parameters:
    -----------
    gmm_labels, kmeans_labels, agg_labels, cpd_labels : np.ndarray
        Labels from each method
    gmm_probs : np.ndarray
        GMM probability matrix (for confidence weighting)
    kmeans_distances : np.ndarray
        K-means distances to cluster centers (for confidence weighting)
    weights : dict
        Base weights for each method
        
    Returns:
    --------
    Tuple[np.ndarray, np.ndarray]
        Ensemble labels and confidence scores
    """
    if weights is None:
        weights = {'gmm': 0.3, 'kmeans': 0.25, 'agg': 0.25, 'cpd': 0.2}
    
    n_samples = len(gmm_labels)
    
    # Find number of unique regimes across all methods
    all_unique = max(
        len(np.unique(gmm_labels)),
        len(np.unique(kmeans_labels)),
        len(np.unique(agg_labels)),
        len(np.unique(cpd_labels))
    )
    
    # Create voting matrix
    ensemble_labels = np.zeros(n_samples, dtype=int)
    confidence_scores = np.zeros(n_samples)
    
    # For each sample, use majority voting
    for i in range(n_samples):
        votes = [
            (gmm_labels[i], weights['gmm']),
            (kmeans_labels[i], weights['kmeans']),
            (agg_labels[i], weights['agg']),
            (cpd_labels[i], weights['cpd'])
        ]
        
        # Aggregate votes by label
        vote_counts = {}
        for label, weight in votes:
            vote_counts[label] = vote_counts.get(label, 0) + weight
        
        # Select winner
        winner = max(vote_counts.keys(), key=lambda x: vote_counts[x])
        ensemble_labels[i] = winner
        
        # Confidence = fraction of weighted votes for winner
        total_weight = sum(weights.values())
        confidence_scores[i] = vote_counts[winner] / total_weight
    
    return ensemble_labels, confidence_scores


def map_regime_labels(labels: np.ndarray, features: pd.DataFrame,
                      feature_names: list = None) -> pd.Series:
    """
    Map numeric cluster labels to meaningful regime names based on feature centroids.
    """
    if feature_names is None:
        feature_names = ['returns_20d', 'vol_20d']
    
    # Calculate centroids
    df = features.copy()
    df['cluster'] = labels
    
    centroids = df.groupby('cluster')[feature_names].mean()
    
    # Define regime mapping based on centroid characteristics
    regime_names = {}
    
    for cluster in centroids.index:
        ret_col = [c for c in feature_names if 'return' in c.lower()]
        vol_col = [c for c in feature_names if 'vol' in c.lower()]
        
        if ret_col and vol_col:
            avg_return = centroids.loc[cluster, ret_col[0]]
            avg_vol = centroids.loc[cluster, vol_col[0]]
            
            # Determine regime name
            if avg_return > 0 and avg_vol < centroids[vol_col[0]].median():
                regime_names[cluster] = 'Bull_Low_Vol'
            elif avg_return > 0 and avg_vol >= centroids[vol_col[0]].median():
                regime_names[cluster] = 'Bull_High_Vol'
            elif avg_return < 0:
                regime_names[cluster] = 'Bear'
            else:
                regime_names[cluster] = 'Choppy'
        else:
            regime_names[cluster] = f'Regime_{cluster}'
    
    return pd.Series([regime_names.get(l, 'Unknown') for l in labels], index=features.index)


print("Ensemble functions defined.")

### 6.4 Run Ensemble Regime Detection on Market Data

In [ ]:
# ============================================================================
# RUN ENSEMBLE REGIME DETECTION - MARKET
# ============================================================================

print("="*60)
print("MARKET REGIME DETECTION (SPY/QQQ)")
print("="*60)

# Run individual methods
gmm_labels_mkt, gmm_model_mkt, gmm_n_mkt, gmm_probs_mkt = fit_gmm_regimes(market_features)
kmeans_labels_mkt, kmeans_model_mkt, kmeans_k_mkt, kmeans_dist_mkt = fit_kmeans_regimes(market_features)
agg_labels_mkt, agg_n_mkt = fit_agglomerative_regimes(market_features)
cpd_labels_mkt, cpd_bkps_mkt = detect_change_points(market_features)

# Ensemble classification
ensemble_labels_mkt, confidence_mkt = ensemble_regime_classification(
    gmm_labels_mkt, kmeans_labels_mkt, agg_labels_mkt, cpd_labels_mkt,
    gmm_probs=gmm_probs_mkt, kmeans_distances=kmeans_dist_mkt
)

# Map to meaningful names
market_regimes = map_regime_labels(ensemble_labels_mkt, market_features, 
                                   ['returns_20d', 'vol_20d'])

print(f"\nMarket Regime Distribution:")
print(market_regimes.value_counts())
print(f"\nAverage Confidence: {confidence_mkt.mean():.2%}")

In [ ]:
# ============================================================================
# RUN ENSEMBLE REGIME DETECTION - SECTOR
# ============================================================================

print("="*60)
print(f"SECTOR REGIME DETECTION ({SECTOR_ETF})")
print("="*60)

# Run individual methods
gmm_labels_sec, gmm_model_sec, gmm_n_sec, gmm_probs_sec = fit_gmm_regimes(sector_features)
kmeans_labels_sec, kmeans_model_sec, kmeans_k_sec, kmeans_dist_sec = fit_kmeans_regimes(sector_features)
agg_labels_sec, agg_n_sec = fit_agglomerative_regimes(sector_features)
cpd_labels_sec, cpd_bkps_sec = detect_change_points(sector_features)

# Ensemble classification
ensemble_labels_sec, confidence_sec = ensemble_regime_classification(
    gmm_labels_sec, kmeans_labels_sec, agg_labels_sec, cpd_labels_sec,
    gmm_probs=gmm_probs_sec, kmeans_distances=kmeans_dist_sec
)

# Map to meaningful names
sector_regimes = map_regime_labels(ensemble_labels_sec, sector_features,
                                   ['returns_20d', 'vol_20d'])

print(f"\nSector Regime Distribution:")
print(sector_regimes.value_counts())
print(f"\nAverage Confidence: {confidence_sec.mean():.2%}")

In [ ]:
# ============================================================================
# RUN ENSEMBLE REGIME DETECTION - ASSET
# ============================================================================

print("="*60)
print(f"ASSET REGIME DETECTION ({TICKER})")
print("="*60)

# Run individual methods
gmm_labels_ast, gmm_model_ast, gmm_n_ast, gmm_probs_ast = fit_gmm_regimes(asset_features)
kmeans_labels_ast, kmeans_model_ast, kmeans_k_ast, kmeans_dist_ast = fit_kmeans_regimes(asset_features)
agg_labels_ast, agg_n_ast = fit_agglomerative_regimes(asset_features)
cpd_labels_ast, cpd_bkps_ast = detect_change_points(asset_features)

# Ensemble classification
ensemble_labels_ast, confidence_ast = ensemble_regime_classification(
    gmm_labels_ast, kmeans_labels_ast, agg_labels_ast, cpd_labels_ast,
    gmm_probs=gmm_probs_ast, kmeans_distances=kmeans_dist_ast
)

# Map to meaningful names
asset_regimes = map_regime_labels(ensemble_labels_ast, asset_features,
                                  ['returns_20d', 'vol_20d'])

print(f"\nAsset Regime Distribution:")
print(asset_regimes.value_counts())
print(f"\nAverage Confidence: {confidence_ast.mean():.2%}")

### 6.5 Regime Validation Metrics

In [ ]:
# ============================================================================
# REGIME VALIDATION
# ============================================================================

def validate_regime_clustering(features: pd.DataFrame, labels: np.ndarray) -> dict:
    """
    Validate clustering quality using multiple metrics.
    """
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(features)
    
    metrics = {
        'silhouette': silhouette_score(X_scaled, labels),
        'davies_bouldin': davies_bouldin_score(X_scaled, labels),
        'calinski_harabasz': calinski_harabasz_score(X_scaled, labels),
        'n_clusters': len(np.unique(labels)),
        'cluster_sizes': dict(zip(*np.unique(labels, return_counts=True)))
    }
    
    return metrics


# Validate all regime detections
print("Regime Validation Metrics:")
print("="*60)

market_validation = validate_regime_clustering(market_features, ensemble_labels_mkt)
print(f"\nMarket Regimes:")
print(f"  Silhouette Score: {market_validation['silhouette']:.3f} (higher is better)")
print(f"  Davies-Bouldin Index: {market_validation['davies_bouldin']:.3f} (lower is better)")
print(f"  Calinski-Harabasz Index: {market_validation['calinski_harabasz']:.1f} (higher is better)")

sector_validation = validate_regime_clustering(sector_features, ensemble_labels_sec)
print(f"\nSector Regimes:")
print(f"  Silhouette Score: {sector_validation['silhouette']:.3f}")
print(f"  Davies-Bouldin Index: {sector_validation['davies_bouldin']:.3f}")
print(f"  Calinski-Harabasz Index: {sector_validation['calinski_harabasz']:.1f}")

asset_validation = validate_regime_clustering(asset_features, ensemble_labels_ast)
print(f"\nAsset Regimes:")
print(f"  Silhouette Score: {asset_validation['silhouette']:.3f}")
print(f"  Davies-Bouldin Index: {asset_validation['davies_bouldin']:.3f}")
print(f"  Calinski-Harabasz Index: {asset_validation['calinski_harabasz']:.1f}")

### 6.6 PRIORITY VISUALIZATION #3: Regime Probability Over Time

In [ ]:
# ============================================================================
# PRIORITY VISUALIZATION #3a: Market Regime Over Time
# ============================================================================

def plot_regime_probabilities(price_data: pd.DataFrame, regime_series: pd.Series,
                              confidence_scores: np.ndarray, cpd_breakpoints: list,
                              title: str, figsize: tuple = (14, 10)):
    """
    Create regime visualization with price, regimes, and confidence.
    """
    fig, axes = plt.subplots(3, 1, figsize=figsize, gridspec_kw={'height_ratios': [2, 1, 1]})
    
    # Align data
    common_idx = price_data.index.intersection(regime_series.index)
    price = price_data.loc[common_idx, 'close']
    regimes = regime_series.loc[common_idx]
    conf = pd.Series(confidence_scores, index=regime_series.index).loc[common_idx]
    
    # Panel 1: Price with regime shading
    ax1 = axes[0]
    ax1.plot(price, color='black', linewidth=1)
    
    # Add regime shading
    prev_regime = None
    segment_start = common_idx[0]
    
    for idx in common_idx:
        current_regime = regimes.loc[idx]
        if current_regime != prev_regime and prev_regime is not None:
            color = get_regime_color(prev_regime)
            ax1.axvspan(segment_start, idx, alpha=0.3, color=color, label=prev_regime)
            segment_start = idx
        prev_regime = current_regime
    
    # Final segment
    if prev_regime:
        color = get_regime_color(prev_regime)
        ax1.axvspan(segment_start, common_idx[-1], alpha=0.3, color=color)
    
    ax1.set_title(title, fontsize=14)
    ax1.set_ylabel('Price')
    ax1.grid(True, alpha=0.3)
    
    # Add legend for regimes
    unique_regimes = regimes.unique()
    patches = [mpatches.Patch(color=get_regime_color(r), alpha=0.5, label=r) 
               for r in unique_regimes]
    ax1.legend(handles=patches, loc='upper left', fontsize=9)
    
    # Panel 2: Regime classification
    ax2 = axes[1]
    regime_numeric = pd.Categorical(regimes).codes
    ax2.plot(common_idx, regime_numeric, drawstyle='steps-post', color='navy', linewidth=1)
    ax2.set_ylabel('Regime')
    ax2.set_yticks(range(len(unique_regimes)))
    ax2.set_yticklabels(unique_regimes)
    ax2.grid(True, alpha=0.3)
    
    # Add CPD breakpoints
    if cpd_breakpoints:
        for bkp in cpd_breakpoints[:-1]:  # Exclude last (end of data)
            if bkp < len(common_idx):
                ax2.axvline(x=common_idx[bkp], color='red', linestyle='--', alpha=0.5)
    
    # Panel 3: Confidence score
    ax3 = axes[2]
    ax3.fill_between(common_idx, conf, alpha=0.5, color='green')
    ax3.plot(common_idx, conf, color='darkgreen', linewidth=1)
    ax3.axhline(y=0.7, color='orange', linestyle='--', label='Min confidence (0.7)')
    ax3.set_ylabel('Confidence')
    ax3.set_xlabel('Date')
    ax3.set_ylim(0, 1)
    ax3.legend(loc='lower right', fontsize=9)
    ax3.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()


# Plot Market Regimes
plot_regime_probabilities(
    df_market, market_regimes, confidence_mkt, cpd_bkps_mkt,
    f'{MARKET_PROXY} - Market Regime Detection (Ensemble Method)',
    figsize=FIG_SIZE_REGIME
)

In [ ]:
# ============================================================================
# PRIORITY VISUALIZATION #3b: Sector Regime Over Time
# ============================================================================

plot_regime_probabilities(
    df_sector, sector_regimes, confidence_sec, cpd_bkps_sec,
    f'{SECTOR_ETF} - Sector Regime Detection (Ensemble Method)',
    figsize=FIG_SIZE_REGIME
)

In [ ]:
# ============================================================================
# PRIORITY VISUALIZATION #3c: Asset Regime Over Time
# ============================================================================

plot_regime_probabilities(
    df_asset, asset_regimes, confidence_ast, cpd_bkps_ast,
    f'{TICKER} - Asset Regime Detection (Ensemble Method)',
    figsize=FIG_SIZE_REGIME
)

### 6.7 Method Agreement Visualization

In [ ]:
# ============================================================================
# METHOD AGREEMENT VISUALIZATION
# ============================================================================

def plot_regime_comparison(gmm_labels, kmeans_labels, agg_labels, cpd_labels,
                           ensemble_labels, index, title):
    """
    Compare regime assignments from each individual method.
    """
    fig, axes = plt.subplots(5, 1, figsize=(14, 10), sharex=True)
    
    methods = [
        ('GMM', gmm_labels),
        ('K-Means', kmeans_labels),
        ('Agglomerative', agg_labels),
        ('CPD', cpd_labels),
        ('Ensemble', ensemble_labels)
    ]
    
    colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd']
    
    for ax, (method, labels), color in zip(axes, methods, colors):
        ax.plot(index, labels, drawstyle='steps-post', color=color, linewidth=1)
        ax.fill_between(index, labels, alpha=0.3, step='post', color=color)
        ax.set_ylabel(method, fontsize=10)
        ax.grid(True, alpha=0.3)
        
        # Highlight ensemble row
        if method == 'Ensemble':
            ax.set_facecolor('#f0f0f0')
    
    axes[0].set_title(title, fontsize=14)
    axes[-1].set_xlabel('Date')
    
    plt.tight_layout()
    plt.show()


# Plot method comparison for market
plot_regime_comparison(
    gmm_labels_mkt, kmeans_labels_mkt, agg_labels_mkt, cpd_labels_mkt,
    ensemble_labels_mkt, market_features.index,
    f'{MARKET_PROXY} - Regime Method Comparison'
)

In [ ]:
# ============================================================================
# CONSENSUS HEATMAP
# ============================================================================

def plot_ensemble_consensus(gmm_labels, kmeans_labels, agg_labels, cpd_labels, index, title):
    """
    Show agreement level between methods over time.
    """
    # Calculate agreement matrix
    n_samples = len(gmm_labels)
    agreement = np.zeros(n_samples)
    
    for i in range(n_samples):
        labels = [gmm_labels[i], kmeans_labels[i], agg_labels[i], cpd_labels[i]]
        # Count most common label
        most_common_count = max(labels.count(l) for l in set(labels))
        agreement[i] = most_common_count / 4  # 4 methods
    
    fig, ax = plt.subplots(figsize=(14, 4))
    
    # Color by agreement level
    colors = np.where(agreement >= 0.75, '#2ecc71',  # High agreement
              np.where(agreement >= 0.5, '#f39c12',  # Moderate
                       '#e74c3c'))  # Low agreement
    
    ax.scatter(index, agreement, c=colors, s=10, alpha=0.7)
    ax.axhline(y=0.75, color='green', linestyle='--', label='High agreement (75%+)')
    ax.axhline(y=0.5, color='orange', linestyle='--', label='Moderate (50%)')
    
    ax.set_title(title, fontsize=14)
    ax.set_xlabel('Date')
    ax.set_ylabel('Agreement Level')
    ax.set_ylim(0, 1.05)
    ax.legend(loc='lower right')
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Print summary
    print(f"Agreement Statistics:")
    print(f"  High agreement (75%+): {(agreement >= 0.75).mean():.1%}")
    print(f"  Moderate (50-75%): {((agreement >= 0.5) & (agreement < 0.75)).mean():.1%}")
    print(f"  Low (<50%): {(agreement < 0.5).mean():.1%}")


plot_ensemble_consensus(
    gmm_labels_mkt, kmeans_labels_mkt, agg_labels_mkt, cpd_labels_mkt,
    market_features.index,
    f'{MARKET_PROXY} - Ensemble Method Agreement Over Time'
)

---

## Section 7: Optimal Strategy Selection & Output

This section synthesizes all analysis to determine optimal strategy allocation by regime.

### 7.1 Strategy Performance by Regime Combination

In [ ]:
# ============================================================================
# SECTION 7: OPTIMAL STRATEGY SELECTION
# ============================================================================

# Create combined regime DataFrame
common_idx = market_regimes.index.intersection(sector_regimes.index).intersection(asset_regimes.index)
common_idx = common_idx.intersection(vol_regime.dropna().index)
common_idx = common_idx.intersection(strategy_returns_df.index)

regime_df = pd.DataFrame({
    'market_regime': market_regimes.loc[common_idx],
    'sector_regime': sector_regimes.loc[common_idx],
    'asset_regime': asset_regimes.loc[common_idx],
    'vol_regime': vol_regime.loc[common_idx],
    'market_confidence': pd.Series(confidence_mkt, index=market_regimes.index).loc[common_idx],
    'sector_confidence': pd.Series(confidence_sec, index=sector_regimes.index).loc[common_idx],
    'asset_confidence': pd.Series(confidence_ast, index=asset_regimes.index).loc[common_idx]
}, index=common_idx)

# Add strategy returns
for strategy in STRATEGIES:
    regime_df[f'{strategy}_return'] = strategy_returns_df[strategy].loc[common_idx]

print(f"Combined regime data: {len(regime_df)} observations")
print(f"\nRegime combinations:")
combo_counts = regime_df.groupby(['market_regime', 'sector_regime']).size()
print(combo_counts.sort_values(ascending=False).head(10))

In [ ]:
# ============================================================================
# STRATEGY PERFORMANCE BY REGIME
# ============================================================================

def calculate_regime_performance(regime_df: pd.DataFrame, strategies: list) -> pd.DataFrame:
    """
    Calculate strategy performance metrics by regime combination.
    """
    results = []
    
    # Group by market and sector regime
    groups = regime_df.groupby(['market_regime', 'sector_regime'])
    
    for (mkt_regime, sec_regime), group in groups:
        for strategy in strategies:
            col = f'{strategy}_return'
            if col in group.columns:
                returns = group[col].dropna()
                
                if len(returns) >= 10:  # Minimum sample size
                    total_return = (1 + returns).prod() - 1
                    avg_return = returns.mean()
                    vol = returns.std()
                    sharpe = (avg_return * 252) / (vol * np.sqrt(252)) if vol > 0 else 0
                    win_rate = (returns > 0).mean()
                    
                    # Drawdown
                    cumulative = (1 + returns).cumprod()
                    running_max = cumulative.expanding().max()
                    max_dd = ((cumulative - running_max) / running_max).min()
                    
                    # Average confidence
                    avg_conf = (group['market_confidence'] + group['sector_confidence']).mean() / 2
                    
                    results.append({
                        'market_regime': mkt_regime,
                        'sector_regime': sec_regime,
                        'strategy': strategy,
                        'avg_return': avg_return,
                        'total_return': total_return,
                        'sharpe': sharpe,
                        'win_rate': win_rate,
                        'max_drawdown': max_dd,
                        'sample_size': len(returns),
                        'confidence': avg_conf
                    })
    
    return pd.DataFrame(results)


regime_performance = calculate_regime_performance(regime_df, STRATEGIES)

print("Strategy Performance by Regime:")
print(regime_performance.sort_values('sharpe', ascending=False).head(20).to_string(index=False))

### 7.2 PRIORITY VISUALIZATION #4: Strategy Performance Heatmaps by Regime

In [ ]:
# ============================================================================
# PRIORITY VISUALIZATION #4: Performance Heatmaps by Strategy
# ============================================================================

def plot_regime_performance_heatmap(perf_df: pd.DataFrame, strategy: str):
    """
    Create heatmap showing strategy performance across regime combinations.
    """
    strategy_data = perf_df[perf_df['strategy'] == strategy]
    
    if strategy_data.empty:
        print(f"No data for {strategy}")
        return
    
    # Pivot for heatmap
    pivot_return = strategy_data.pivot(index='market_regime', columns='sector_regime', 
                                        values='avg_return').fillna(0)
    pivot_winrate = strategy_data.pivot(index='market_regime', columns='sector_regime',
                                         values='win_rate').fillna(0)
    pivot_sample = strategy_data.pivot(index='market_regime', columns='sector_regime',
                                        values='sample_size').fillna(0)
    
    fig, ax = plt.subplots(figsize=FIG_SIZE_HEATMAP)
    
    # Main heatmap with returns
    color = get_strategy_color(strategy)
    
    # Use diverging colormap
    vmax = max(abs(pivot_return.values.min()), abs(pivot_return.values.max()))
    vmin = -vmax
    
    sns.heatmap(pivot_return * 100, annot=True, fmt='.2f', cmap='RdYlGn',
                ax=ax, center=0, vmin=vmin*100, vmax=vmax*100,
                linewidths=0.5, cbar_kws={'label': 'Avg Daily Return (%)'})
    
    # Add sample size annotations
    for i in range(len(pivot_return.index)):
        for j in range(len(pivot_return.columns)):
            n = pivot_sample.iloc[i, j]
            wr = pivot_winrate.iloc[i, j]
            if n > 0:
                ax.annotate(f'n={int(n)}\nWR={wr:.0%}',
                           xy=(j + 0.5, i + 0.7),
                           ha='center', va='center',
                           fontsize=8, alpha=0.7)
    
    ax.set_title(f'{strategy} - Performance by Regime Combination', 
                 fontsize=14, color=color, fontweight='bold')
    ax.set_xlabel('Sector Regime')
    ax.set_ylabel('Market Regime')
    
    plt.tight_layout()
    plt.show()


# Create separate heatmaps for each strategy
for strategy in STRATEGIES:
    plot_regime_performance_heatmap(regime_performance, strategy)
    print()

### 7.3 Optimal Strategy Selection by Regime

In [ ]:
# ============================================================================
# OPTIMAL STRATEGY BY REGIME
# ============================================================================

def find_optimal_strategy(perf_df: pd.DataFrame) -> pd.DataFrame:
    """
    Find the optimal strategy for each regime combination.
    """
    # For each regime combination, find best strategy by Sharpe
    optimal = []
    
    groups = perf_df.groupby(['market_regime', 'sector_regime'])
    
    for (mkt, sec), group in groups:
        if len(group) > 0:
            best = group.loc[group['sharpe'].idxmax()]
            
            # Also find optimal stop loss (simplified - use best overall for now)
            stop_data = stop_loss_df[stop_loss_df['strategy'] == best['strategy']]
            if len(stop_data) > 0:
                best_stop = stop_data.loc[stop_data['sharpe'].idxmax(), 'stop_loss']
            else:
                best_stop = 'None'
            
            optimal.append({
                'Regime_Market': mkt,
                'Regime_Sector': sec,
                'Recommended_Strategy': best['strategy'],
                'Expected_Return': best['avg_return'],
                'Sharpe': best['sharpe'],
                'Max_DD': best['max_drawdown'],
                'Win_Rate': best['win_rate'],
                'Sample_Size': best['sample_size'],
                'Ensemble_Confidence': best['confidence'],
                'Stop_Loss_%': best_stop
            })
    
    return pd.DataFrame(optimal)


optimal_strategies = find_optimal_strategy(regime_performance)

print("OPTIMAL STRATEGY ALLOCATION BY REGIME:")
print("="*80)
print(optimal_strategies.to_string(index=False))

In [ ]:
# ============================================================================
# COMPREHENSIVE OUTPUT TABLE
# ============================================================================

# Add volatility regime and asset regime for complete picture
def create_comprehensive_recommendations() -> pd.DataFrame:
    """
    Create comprehensive strategy recommendations table.
    """
    # Get unique regime combinations
    combos = regime_df.groupby(['market_regime', 'sector_regime', 'asset_regime', 'vol_regime']).size()
    combos = combos[combos >= 5].reset_index(name='sample_size')  # Min 5 days
    
    recommendations = []
    
    for _, row in combos.iterrows():
        mkt, sec, ast, vol = row['market_regime'], row['sector_regime'], row['asset_regime'], row['vol_regime']
        
        # Filter data for this regime combination
        mask = (
            (regime_df['market_regime'] == mkt) &
            (regime_df['sector_regime'] == sec) &
            (regime_df['asset_regime'] == ast) &
            (regime_df['vol_regime'] == vol)
        )
        subset = regime_df[mask]
        
        if len(subset) < 5:
            continue
        
        # Find best strategy
        best_strategy = None
        best_sharpe = -np.inf
        best_return = 0
        best_winrate = 0
        
        for strategy in STRATEGIES:
            col = f'{strategy}_return'
            if col in subset.columns:
                returns = subset[col].dropna()
                if len(returns) >= 5:
                    avg_ret = returns.mean()
                    vol_ret = returns.std()
                    sharpe = (avg_ret * 252) / (vol_ret * np.sqrt(252)) if vol_ret > 0 else 0
                    win_rate = (returns > 0).mean()
                    
                    if sharpe > best_sharpe:
                        best_sharpe = sharpe
                        best_strategy = strategy
                        best_return = avg_ret
                        best_winrate = win_rate
        
        if best_strategy:
            # Determine delta based on strategy type
            call_delta = 'N/A'
            put_delta = 'N/A'
            
            if 'Call' in best_strategy:
                call_delta = '0.40'
            if 'Put' in best_strategy:
                put_delta = '-0.20'
            if best_strategy in ['Straddle', 'Strangle']:
                call_delta = '0.30'
                put_delta = '-0.30'
            if 'Calendar' in best_strategy or 'Diagonal' in best_strategy:
                call_delta = '0.50'
            
            # Get recommended stop loss
            stop_data = stop_loss_df[stop_loss_df['strategy'] == best_strategy]
            if len(stop_data) > 0:
                stop_loss = stop_data.loc[stop_data['sharpe'].idxmax(), 'stop_loss']
            else:
                stop_loss = 'None'
            
            # Average confidence
            avg_conf = subset[['market_confidence', 'sector_confidence', 'asset_confidence']].mean().mean()
            
            recommendations.append({
                'Regime_Market': mkt,
                'Regime_Sector': sec,
                'Regime_Asset': ast,
                'Volatility_Regime': vol,
                'Ensemble_Confidence': f'{avg_conf:.2f}',
                'Recommended_Strategy': best_strategy,
                'Call_Delta': call_delta,
                'Put_Delta': put_delta,
                'Stop_Loss_%': stop_loss,
                'Expected_Return': f'{best_return*100:.2f}%',
                'Sharpe': f'{best_sharpe:.2f}',
                'Win_Rate': f'{best_winrate:.0%}',
                'Sample_Size': len(subset)
            })
    
    return pd.DataFrame(recommendations)


comprehensive_recommendations = create_comprehensive_recommendations()

print("\nCOMPREHENSIVE STRATEGY RECOMMENDATIONS:")
print("="*120)
print(comprehensive_recommendations.to_string(index=False))

# Save to CSV
# comprehensive_recommendations.to_csv('strategy_recommendations.csv', index=False)
# print("\nRecommendations saved to strategy_recommendations.csv")

### 7.4 Current Regime Analysis

In [ ]:
# ============================================================================
# CURRENT REGIME ANALYSIS
# ============================================================================

# Get current (most recent) regime states
current_market = market_regimes.iloc[-1]
current_sector = sector_regimes.iloc[-1]
current_asset = asset_regimes.iloc[-1]
current_vol = vol_regime.iloc[-1] if len(vol_regime) > 0 else 'Unknown'

current_market_conf = confidence_mkt[-1] if len(confidence_mkt) > 0 else 0
current_sector_conf = confidence_sec[-1] if len(confidence_sec) > 0 else 0
current_asset_conf = confidence_ast[-1] if len(confidence_ast) > 0 else 0

print("CURRENT REGIME STATES:")
print("="*60)
print(f"Date: {regime_df.index[-1]}")
print(f"\nMarket Regime: {current_market} (Confidence: {current_market_conf:.2%})")
print(f"Sector Regime: {current_sector} (Confidence: {current_sector_conf:.2%})")
print(f"Asset Regime:  {current_asset} (Confidence: {current_asset_conf:.2%})")
print(f"Volatility:    {current_vol}")

# Find recommended strategy for current regime
current_rec = comprehensive_recommendations[
    (comprehensive_recommendations['Regime_Market'] == current_market) &
    (comprehensive_recommendations['Regime_Sector'] == current_sector)
]

if len(current_rec) > 0:
    print("\nRECOMMENDED STRATEGY FOR CURRENT REGIME:")
    print("-"*60)
    rec = current_rec.iloc[0]
    print(f"Strategy: {rec['Recommended_Strategy']}")
    print(f"Call Delta: {rec['Call_Delta']}")
    print(f"Put Delta: {rec['Put_Delta']}")
    print(f"Stop Loss: {rec['Stop_Loss_%']}")
    print(f"Expected Return: {rec['Expected_Return']}")
    print(f"Sharpe Ratio: {rec['Sharpe']}")
    print(f"Win Rate: {rec['Win_Rate']}")
else:
    print("\nNo historical data for current regime combination.")
    print("Consider using the most similar regime or conservative positioning.")

### 7.5 PRIORITY VISUALIZATION #5: Summary Dashboard

In [ ]:
# ============================================================================
# PRIORITY VISUALIZATION #5: Summary Dashboard
# ============================================================================

fig = plt.figure(figsize=FIG_SIZE_DASHBOARD)

# Create 3x3 grid
gs = fig.add_gridspec(3, 3, hspace=0.3, wspace=0.3)

# Panel 1: Current regime states
ax1 = fig.add_subplot(gs[0, 0])
regimes = ['Market', 'Sector', 'Asset']
regime_values = [current_market, current_sector, current_asset]
confidence_values = [current_market_conf, current_sector_conf, current_asset_conf]
colors = [get_regime_color(r) for r in regime_values]

bars = ax1.barh(regimes, confidence_values, color=colors, alpha=0.7)
ax1.set_xlim(0, 1)
ax1.set_xlabel('Confidence')
ax1.set_title('Current Regime States', fontsize=12)

for bar, val, regime in zip(bars, confidence_values, regime_values):
    ax1.annotate(f'{regime}\n{val:.0%}', 
                xy=(val + 0.02, bar.get_y() + bar.get_height()/2),
                va='center', fontsize=9)

# Panel 2: Volatility distribution
ax2 = fig.add_subplot(gs[0, 1])
vol_hist = realized_vol['rv_true'].dropna()
current_vol_value = vol_hist.iloc[-1] if len(vol_hist) > 0 else 0

ax2.hist(vol_hist, bins=30, alpha=0.7, color='#3498db', edgecolor='black')
ax2.axvline(x=current_vol_value, color='red', linewidth=2, label=f'Current: {current_vol_value:.2%}')
ax2.axvline(x=vol_hist.mean(), color='green', linestyle='--', label=f'Mean: {vol_hist.mean():.2%}')
ax2.set_title('Volatility Distribution', fontsize=12)
ax2.set_xlabel('Volatility')
ax2.legend(fontsize=8)

# Panel 3: Recommended strategy
ax3 = fig.add_subplot(gs[0, 2])
ax3.axis('off')

if len(current_rec) > 0:
    rec = current_rec.iloc[0]
    strategy_color = get_strategy_color(rec['Recommended_Strategy'])
    
    text = f"RECOMMENDED STRATEGY\n\n"
    text += f"{rec['Recommended_Strategy']}\n\n"
    text += f"Stop Loss: {rec['Stop_Loss_%']}\n"
    text += f"Expected Return: {rec['Expected_Return']}\n"
    text += f"Sharpe: {rec['Sharpe']}"
    
    ax3.text(0.5, 0.5, text, ha='center', va='center', fontsize=11,
             transform=ax3.transAxes,
             bbox=dict(boxstyle='round', facecolor=strategy_color, alpha=0.3))
else:
    ax3.text(0.5, 0.5, 'No recommendation\nfor current regime', 
             ha='center', va='center', fontsize=11, transform=ax3.transAxes)

ax3.set_title('Strategy Recommendation', fontsize=12)

# Panel 4: Performance metrics table
ax4 = fig.add_subplot(gs[1, 0])
ax4.axis('off')

if len(current_rec) > 0:
    rec = current_rec.iloc[0]
    table_data = [
        ['Metric', 'Value'],
        ['Win Rate', rec['Win_Rate']],
        ['Sharpe', rec['Sharpe']],
        ['Sample Size', str(rec['Sample_Size'])],
        ['Confidence', rec['Ensemble_Confidence']]
    ]
    
    table = ax4.table(cellText=table_data, loc='center', cellLoc='center',
                      colWidths=[0.4, 0.4])
    table.auto_set_font_size(False)
    table.set_fontsize(10)
    table.scale(1.2, 1.5)

ax4.set_title('Expected Performance', fontsize=12)

# Panel 5: Historical performance of recommended strategy
ax5 = fig.add_subplot(gs[1, 1])

if len(current_rec) > 0:
    rec_strategy = current_rec.iloc[0]['Recommended_Strategy']
    strategy_returns_series = strategy_returns_df[rec_strategy]
    cumulative = (1 + strategy_returns_series).cumprod()
    
    color = get_strategy_color(rec_strategy)
    ax5.plot(cumulative, color=color, linewidth=1.5)
    ax5.set_title(f'{rec_strategy} Historical Performance', fontsize=12)
    ax5.set_ylabel('Cumulative Return')
    ax5.grid(True, alpha=0.3)

# Panel 6: Risk warnings
ax6 = fig.add_subplot(gs[1, 2])
ax6.axis('off')

# Check for warnings
warnings = []
avg_conf = (current_market_conf + current_sector_conf + current_asset_conf) / 3

if avg_conf < 0.6:
    warnings.append("⚠️ Low regime confidence")
if current_vol == 'Extreme':
    warnings.append("⚠️ Extreme volatility detected")
if len(current_rec) == 0:
    warnings.append("⚠️ Novel regime combination")
if len(current_rec) > 0 and int(current_rec.iloc[0]['Sample_Size']) < 20:
    warnings.append("⚠️ Limited historical data")

if warnings:
    warning_text = "RISK WARNINGS\n\n" + "\n".join(warnings)
    ax6.text(0.5, 0.5, warning_text, ha='center', va='center', fontsize=10,
             transform=ax6.transAxes,
             bbox=dict(boxstyle='round', facecolor='#ffcccc', alpha=0.5))
else:
    ax6.text(0.5, 0.5, '✅ No significant warnings\n\nRegime stable with\nhigh confidence',
             ha='center', va='center', fontsize=10, transform=ax6.transAxes,
             bbox=dict(boxstyle='round', facecolor='#ccffcc', alpha=0.5))

ax6.set_title('Risk Alerts', fontsize=12)

# Panel 7: Method agreement pie chart
ax7 = fig.add_subplot(gs[2, 0])

# Calculate method agreement
agreement_levels = {
    'High (75%+)': (confidence_mkt >= 0.75).mean(),
    'Moderate (50-75%)': ((confidence_mkt >= 0.5) & (confidence_mkt < 0.75)).mean(),
    'Low (<50%)': (confidence_mkt < 0.5).mean()
}

colors_pie = ['#2ecc71', '#f39c12', '#e74c3c']
ax7.pie([v for v in agreement_levels.values()], labels=agreement_levels.keys(),
        autopct='%1.0f%%', colors=colors_pie, startangle=90)
ax7.set_title('Ensemble Agreement Distribution', fontsize=12)

# Panel 8: PCA feature space (simplified 2D)
ax8 = fig.add_subplot(gs[2, 1])

# Quick PCA for visualization
scaler_viz = StandardScaler()
X_viz = scaler_viz.fit_transform(asset_features.dropna())
pca_viz = PCA(n_components=2)
X_pca = pca_viz.fit_transform(X_viz)

# Color by regime
colors_scatter = [get_regime_color(r) for r in asset_regimes.loc[asset_features.dropna().index]]
ax8.scatter(X_pca[:, 0], X_pca[:, 1], c=colors_scatter, alpha=0.5, s=10)
ax8.scatter(X_pca[-1, 0], X_pca[-1, 1], c='black', s=100, marker='*', 
            label='Current', edgecolors='white', linewidth=1)
ax8.set_xlabel(f'PC1 ({pca_viz.explained_variance_ratio_[0]:.0%})')
ax8.set_ylabel(f'PC2 ({pca_viz.explained_variance_ratio_[1]:.0%})')
ax8.set_title('Feature Space Position', fontsize=12)
ax8.legend(loc='upper right', fontsize=9)
ax8.grid(True, alpha=0.3)

# Panel 9: Regime transition probabilities
ax9 = fig.add_subplot(gs[2, 2])

# Get transition probabilities from current regime
if current_vol in transition_probs.index:
    trans = transition_probs.loc[current_vol]
    colors_bar = [get_vol_regime_color(r) for r in trans.index]
    ax9.bar(trans.index, trans.values, color=colors_bar, alpha=0.7)
    ax9.set_ylim(0, 1)
    ax9.set_ylabel('Probability')
    ax9.set_title(f'Next Regime Probability\n(from {current_vol})', fontsize=12)
    ax9.grid(True, alpha=0.3, axis='y')
else:
    ax9.text(0.5, 0.5, 'Transition data\nnot available', 
             ha='center', va='center', transform=ax9.transAxes)
    ax9.set_title('Regime Transition', fontsize=12)

plt.suptitle(f'{TICKER} - Volatility Trading Dashboard', fontsize=16, fontweight='bold', y=0.98)
plt.tight_layout()
plt.show()

### 7.6 Portfolio Extension Framework

In [ ]:
# ============================================================================
# PORTFOLIO EXTENSION FRAMEWORK
# ============================================================================

# This section provides structure for future multi-asset portfolio optimization

class VolatilityTradingPortfolio:
    """
    Framework for multi-asset volatility trading portfolio.
    
    Designed to be extended for portfolio-level optimization.
    """
    
    def __init__(self, tickers: List[str]):
        self.tickers = tickers
        self.asset_data = {}
        self.regime_data = {}
        self.strategy_allocations = {}
        
    def add_asset(self, ticker: str, data: pd.DataFrame, regimes: pd.Series):
        """Add asset data to portfolio."""
        self.asset_data[ticker] = data
        self.regime_data[ticker] = regimes
        
    def calculate_cross_asset_correlation(self) -> pd.DataFrame:
        """Calculate correlation matrix across assets."""
        returns_df = pd.DataFrame()
        for ticker, data in self.asset_data.items():
            returns_df[ticker] = data['returns'] if 'returns' in data.columns else data['close'].pct_change()
        return returns_df.corr()
    
    def calculate_portfolio_greeks(self, positions: dict) -> dict:
        """
        Calculate aggregate portfolio Greeks.
        
        Parameters:
        -----------
        positions : dict
            Dictionary of position sizes by ticker and strategy
            
        Returns:
        --------
        dict
            Aggregate delta, gamma, theta, vega
        """
        # Placeholder for portfolio-level Greek calculations
        return {
            'portfolio_delta': 0,
            'portfolio_gamma': 0,
            'portfolio_theta': 0,
            'portfolio_vega': 0
        }
    
    def check_regime_correlation(self) -> pd.DataFrame:
        """Check if assets share regimes simultaneously."""
        regime_df = pd.DataFrame()
        for ticker, regimes in self.regime_data.items():
            regime_df[ticker] = pd.Categorical(regimes).codes
        return regime_df.corr()
    
    def optimize_allocation(self, constraints: dict = None) -> dict:
        """
        Optimize strategy allocation across assets.
        
        Placeholder for future implementation of:
        - Mean-variance optimization
        - Risk parity
        - Regime-based allocation
        """
        # Placeholder - to be implemented
        return {ticker: {'strategy': None, 'weight': 0} for ticker in self.tickers}


# Example usage (for future expansion)
print("Portfolio Extension Framework initialized.")
print("")
print("Key methods for future implementation:")
print("  - add_asset(): Add multiple assets to portfolio")
print("  - calculate_cross_asset_correlation(): Diversification analysis")
print("  - calculate_portfolio_greeks(): Aggregate risk exposure")
print("  - check_regime_correlation(): Regime synchronization")
print("  - optimize_allocation(): Multi-asset strategy allocation")

---

## Summary & Conclusions

This notebook has implemented a comprehensive volatility trading research framework including:

### Key Components:
1. **Volatility Modeling**: Multiple models (Historical, EWMA, Parkinson, Garman-Klass, Rogers-Satchell, ATR) with PCA-based aggregate forecasting

2. **Mean Reversion Analysis**: Volatility regime classification with transition probabilities and DTE recommendations

3. **Options Strategies**: Six strategies (Calendar Spread, Double Diagonal, Straddle, Strangle, Bull Put Spread, Bull Call Spread) with delta optimization

4. **Ensemble Regime Detection**: Combined GMM, K-Means, Agglomerative Clustering, and Change-Point Detection for robust regime identification at Market, Sector, and Asset levels

5. **Stop Loss Optimization**: Systematic testing of stop loss thresholds by strategy

6. **Comprehensive Output**: Machine-readable recommendations table with regime-specific strategy allocation

### Technical Implementation:
- All computations use vectorized operations (NumPy/Pandas)
- No multiprocessing (QuantConnect compatible)
- Ticker-agnostic design for easy portfolio extension
- Consistent color palettes across all visualizations

### Next Steps:
1. Connect to live QuantConnect options data
2. Implement actual options pricing models
3. Extend to multi-asset portfolio optimization
4. Add real-time regime monitoring
5. Backtest with actual transaction costs

In [ ]:
# ============================================================================
# END OF NOTEBOOK
# ============================================================================

print("="*60)
print("VOLATILITY TRADING RESEARCH NOTEBOOK COMPLETE")
print("="*60)
print(f"")
print(f"Ticker analyzed: {TICKER}")
print(f"Date range: {START_DATE.date()} to {END_DATE.date()}")
print(f"")
print(f"Current Regimes:")
print(f"  Market: {current_market}")
print(f"  Sector: {current_sector}")
print(f"  Asset:  {current_asset}")
print(f"  Volatility: {current_vol}")
print(f"")

if len(current_rec) > 0:
    rec = current_rec.iloc[0]
    print(f"Recommended Strategy: {rec['Recommended_Strategy']}")
    print(f"Stop Loss: {rec['Stop_Loss_%']}")
else:
    print("No specific recommendation for current regime.")

print(f"")
print("See comprehensive_recommendations DataFrame for full regime-strategy mapping.")